In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
pd.set_option("display.max_columns", 100)
import os
import geojson
from geojson import Feature, FeatureCollection
import requests
from bs4 import BeautifulSoup
import json
import requests
from datetime import datetime
from geopy import geocoders
import random
from datetime import timedelta

In [3]:
folder = "09052020"
today_date = pd.datetime.today()

In [1409]:
root_path = "/Users/nesara/Documents/aim/git/articles/covid19/data/"+folder
if not os.path.exists(root_path):
    os.makedirs(root_path)

In [127]:
"""
url = "https://api.covid19india.org/csv/latest/raw_data1.csv"
r = requests.get(url)
f = open('../data/'+folder+'/raw_data1.csv','w')
f.write(r.text)
f.close()
url = "https://api.covid19india.org/csv/latest/raw_data2.csv"
r = requests.get(url)
f = open('../data/'+folder+'/raw_data2.csv','w')
f.write(r.text)
f.close()
url = "https://api.covid19india.org/csv/latest/death_and_recovered1.csv"
r = requests.get(url)
f = open('../data/'+folder+'/death_and_recovered1.csv','w')
f.write(r.text)
f.close()

url = "https://api.covid19india.org/csv/latest/death_and_recovered2.csv"
r = requests.get(url)
f = open('../data/'+folder+'/death_and_recovered2.csv','w')
f.write(r.text)
f.close()
"""

"""
url = "https://api.covid19india.org/csv/latest/raw_data3.csv"
r = requests.get(url)
f = open('../data/'+folder+'/raw_data3.csv','w')
f.write(r.text)
f.close()
"""

url = "https://api.covid19india.org/csv/latest/state_wise_daily.csv"
r = requests.get(url)
f = open('../data/'+folder+'/statewise_daily_confirmed_deceased_recovered.csv','w')
f.write(r.text)
f.close()

### Separate confirmed, deceased, recovered
df = pd.read_csv('../data/'+folder+'/statewise_daily_confirmed_deceased_recovered.csv')
df.rename(columns={"Date":"date"}, inplace=True)
df.to_csv('../data/'+folder+'/statewise_daily_confirmed_deceased_recovered.csv', index=False)
dft = df[df["Status"]=="Confirmed"].copy()
del dft["Status"]
dft.to_csv('../data/'+folder+'/statewise_daily_confirmed.csv', index=False)
dft = df[df["Status"]=="Deceased"].copy()
del dft["Status"]
dft.to_csv('../data/'+folder+'/statewise_daily_deceased.csv', index=False)
dft = df[df["Status"]=="Recovered"].copy()
del dft["Status"]
dft.to_csv('../data/'+folder+'/statewise_daily_recovered.csv', index=False)


##### Statewise testing numbers #########
url = "https://api.covid19india.org/csv/latest/statewise_tested_numbers_data.csv"
r = requests.get(url)
f = open('../data/'+folder+'/statewise_testing.csv','w')
f.write(r.text)
f.close()

### Country level Daily and overall cases, deaths, recovered
url = "https://api.covid19india.org/csv/latest/case_time_series.csv"
r = requests.get(url)
f = open('../data/'+folder+'/case_time_series.csv','w')
f.write(r.text)
f.close()

## State wise current count
url = "https://api.covid19india.org/csv/latest/state_wise.csv"
r = requests.get(url)
f = open('../data/'+folder+'/state_wise_current.csv','w')
f.write(r.text)
f.close()

In [110]:
map_to_wiki_district_name_mapping = {"Raigarh(MH)":"Raigad", "Raigarh(CG)":"Raigarh",
    "West District":"West Sikkim","East District":"East Sikkim",
    "North  District":"North Sikkim","South District":"South Sikkim",
    "Sipahijala":"Sepahijala", "Subarnapur":"Subarnapur (Sonepur)","Jalor":"Jalore",
    "Jagatsinghapur":"Jagatsinghpur","Nabarangapur":"Nabarangpur",
    "Bargarh":"Bargarh (Baragarh)","Baudh":"Boudh (Bauda)", "Boudh":"Boudh (Bauda)",
    "Anugul":"Angul", "Ribhoi":"Ri Bhoi","Narsimhapur":"Narsinghpur",
    "Mumbai Suburban":"Mumbai suburban", "Shivamogga":"Shimoga", "Chamarajanagara":"Chamarajnagar", "Punch":"Poonch", "The Dangs":"Dang","Lahul and Spiti":"Lahaul and Spiti",
    "Pashchimi Singhbhum":"West Singhbhum","Purbi Singhbhum":"East Singhbhum","Saraikela-kharsawan":"Seraikela Kharsawan", "Saraikela-Kharsawan":"Seraikela Kharsawan",
    "Uttar Bastar Kanker":"Kanker","Devbhumi Dwarka":"Devbhoomi Dwarka",
    "Medinipur East":"Purba Medinipur", "Medinipur West":"Paschim Medinipur",
    "S.P.S. Nellore":"Sri Potti Sriramulu Nellore", "Y.S.R.":"Kadapa", "South Salmara Mancachar":"South Salmara",
    "Kaimur Bhabhua":"Kaimur","Kabeerdham":"Kabirdham", "Ahmadabad":"Ahmedabad", "Banas Kantha":"Banaskantha",
    "Chota Udaipur":"Chhota Udepur", "Dohad":"Dahod", "Mahesana":"Mehsana", "Panch Mahals":"Panchmahal",
    "Sabar Kantha":"Sabarkantha", "Kachchh":"Kutch","Charki Dadri":"Charkhi Dadri", "Gurugram":"Gurgaon",
    "Hisar":"Hissar", "Yamunanagar":"Yamuna Nagar","Bandipore":"Bandipora", "Baramula":"Baramulla",
    "Shupiyan":"Shopian", "Budgam":"Badgam","Hazaribagh":"Hazaribag","Kodarma":"Koderma","Bagalkote":"Bagalkot",
    "Belagavi":"Belgaum", "Bengaluru":"Bangalore Urban", "Bengaluru Rural":"Bangalore Rural",
    "Chikkaballapura":"Chikkaballapur", "Kalaburagi":"Gulbarga","Khandwa":"Khandwa (East Nimar)",
    "Khargone":"Khargone (West Nimar)", "Ashoknagar":"Ashok Nagar","Ahmadnagar":"Ahmednagar","Bid":"Beed",
    "Buldana":"Buldhana","Gondiya":"Gondia","Mumbai":"Mumbai City","Baleshwar":"Balasore","Jajapur":"Jajpur",
    "Debagarh":"Debagarh (Deogarh)", "Kendujhar":"Kendujhar (Keonjhar)", "Mahe":"Mahé", "Puducherry":"Pondicherry",
    "S.A.S. Nagar":"Sahibzada Ajit Singh Nagar", "Dhaulpur":"Dholpur", "Chittaurgarh":"Chittorgarh",
    "Kancheepuram":"Kanchipuram", "Kanniyakumari":"Kanyakumari", "The Nilgiris":"Nilgiris",
    "Thiruvallur":"Tiruvallur", "Thiruvarur":"Tiruvarur", "Tirupathur":"Tirupattur", "Tiruppur":"Tirupur",
    "Jagitial":"Jagtial", "Jayashankar":"Jayashankar Bhupalpally", "Jangoan":"Jangaon", "Kumuram Bheem Asifabad":"Komaram Bheem",
    "Mahabubnagar":"Mahbubnagar", "Medchal Malkajgiri":"Medchal-Malkajgiri", "Baghpat":"Bagpat",
    "Bara Banki":"Barabanki", "Gautam Buddha Nagar":"Gautam Buddh Nagar", "Kheri":"Lakhimpur Kheri", "Mahrajganj":"Maharajganj", "Prayagraj":"Allahabad",
    "Rae Bareli":"Raebareli", "Shrawasti":"Shravasti","Nicobars":"Nicobar", "Biswanath":"Bishwanath",
    "Bametara":"Bemetara", "Dakshin Bastar Dantewada":"Dantewada", "Janjgir Champa":"Janjgir-Champa",
    "North District": "North Sikkim"}


data_to_map_district_name_mapping = {"Jagtial":"Jagitial", "Jangaon":"Jangoan", "Jayashankar Bhupalapally":"Jayashankar",
                            "Komaram Bheem":"Kumuram Bheem Asifabad","Barabanki":"Bara Banki",
                            "Maharajganj":"Mahrajganj", "Ayodhya":"Faizabad", "Lakhimpur Kheri":"Kheri",
                            "Malda":"Maldah", "Paschim Medinipur":"Medinipur West", "Purba Medinipur":"Medinipur East",
                            "South Salmara Mankachar": "South Salmara Mancachar",
                           "Charkhi Dadri":"Charki Dadri", "Ahmednagar":"Ahmadnagar", "Buldhana":"Buldana",
                           "Gondia":"Gondiya", "Beed":"Bid", "Chittorgarh":"Chittaurgarh",
                           "Dholpur":"Dhaulpur", "Jalore":"Jalor", "Y.S.R. Kadapa":"Y.S.R.", "Y.S.R Kadapa":"Y.S.R.",
                           "Kaimur":"Kaimur Bhabhua", "Ahmedabad":"Ahmadabad", "Banaskantha":"Banas Kantha",
                            "Chhota Udaipur":"Chota Udaipur", "Dahod":"Dohad", "Kutch":"Kachchh", "Mehsana":"Mahesana",
                            "Panchmahal":"Panch Mahals", "Sabarkantha":"Sabar Kantha", "Bandipora":"Bandipore",
                            "Baramulla":"Baramula", "Shopiyan":"Shupiyan","Koderma":"Kodarma",
                            "Bengaluru Urban":"Bengaluru", "Balasore":"Baleshwar", "Deogarh":"Debagarh",
                            "Jagatsinghpur":"Jagatsinghapur", "Jajpur":"Jajapur","Ferozepur":"Firozpur",
                            "Kanyakumari":"Kanniyakumari", "Thoothukkudi":"Thoothukudi"}

data_to_map_state_name_mapping = {"Dadra and Nagar Haveli and Daman and Diu":"Dadra and Nagar Haveli"}

district_list_not_in_map = {"Dang", "Budgam",'Chengalpattu' 'Kallakurichi' 'Nilgiris' 'Ranipet' 'Tenkasi' 'Tirupathur',
                           }


### District wise latest confirmed, deaths, active, recovered

In [86]:
url = "https://api.covid19india.org/v2/state_district_wise.json"
r = requests.get(url)
json_data = r.json()

In [114]:
dfmd = pd.read_csv("../data/district_list_map.csv")
dfsc = pd.read_csv("../data/extracted_data_api/state_codes.csv")

dfd = pd.DataFrame()
for i in range(len(json_data)):
    state_name = json_data[i]["state"]
    if state_name in data_to_map_state_name_mapping:
        state_name = data_to_map_state_name_mapping[state_name]
    state_code = json_data[i]["statecode"]
    
    dft = pd.DataFrame(json_data[i]["districtData"])
    dft["state"] = state_name 
    dft["state_code"] = state_code 
    dft = dft[["district", "state", "state_code", "confirmed", "active", "recovered", "deceased"]]
    
    
    ### Map district names
    for j in range(len(dft)):
        district_name = dft.iloc[j]["district"]
        if district_name in data_to_map_district_name_mapping:
            district_name_mapped = data_to_map_district_name_mapping[district_name]
            dft.at[(dft["district"]==district_name) & (dft["state"]==state_name), "district"] = district_name_mapped
            district_name = district_name_mapped
        
        if district_name in district_name_resolution:
            district_name_mapped = district_name+"("+state_code+")"
            dft.at[(dft["district"]==district_name) & (dft["state"]==state_name), "district"] = district_name_mapped
            district_name = district_name_mapped
        
    dft.at[(dft["district"]=="Raigad")&(dft["state"]=="Maharashtra"), "district"] = "Raigarh"
    
    ### Include districts with 0 cases
    dfs = dfmd[dfmd["state"]==state_name]
    dft = dft.set_index(["district", "state"]).join(dfs.set_index(["district", "state"]), how="outer").reset_index()
    dft["state_code"] = state_code
    dft.fillna(0, inplace=True)
    
    dft.at[(dft["district"]=="Raigarh")&(dft["state"]=="Chhattisgarh"), "district"] = "Raigarh(CG)"
    dft.at[(dft["district"]=="Raigarh")&(dft["state"]=="Maharashtra"), "district"] = "Raigarh(MH)"
    
    #full_district_list = dft["district"]
    #map_district_list = dfs["district"]
    #print(np.setdiff1d(full_district_list, map_district_list))
    
    if i==0:
        dfd = dft.copy()
    else:
        dfd = dfd.append(dft)


dfdl = pd.DataFrame()
dfdl.at[0, "district"] = "Delhi"
dfdl.at[0, "state"] = "Delhi"
dfdl.at[0, "state_code"] = "DL"
dftmp = dfd[dfd["state"]=="Delhi"].sum(numeric_only=True)
dfdl.at[0, "confirmed"] = dftmp["confirmed"]
dfdl.at[0, "active"] = dftmp["active"]
dfdl.at[0, "recovered"] = dftmp["recovered"]
dfdl.at[0, "deceased"] = dftmp["deceased"]

dfd = dfd[dfd["state"]!="Delhi"]
dfd = dfd.append(dfdl)

dfd = dfd[dfd["district"]!="Other state"]
dfd = dfd[dfd["district"]!="Other State"]
dfd = dfd[dfd["district"]!="Other States"]
dfd = dfd[dfd["district"]!="Unknown"]
dfd = dfd[dfd["district"]!="Italians"]
dfd = dfd[dfd["district"]!="Evacuees"]
dfd = dfd[dfd["district"]!="BSF Camp"]

unlisted_states = np.setdiff1d(dfmd["state"].unique(), dfd["state"].unique())
for st in unlisted_states:
    dft = dfmd[dfmd["state"]==st].copy()
    state_code = dfsc[dfsc["state"]==st]["state_code"].values[0]
    dft["state_code"] = state_code
    dft["confirmed"] = 0
    dft["active"] = 0
    dft["recovered"] = 0
    dft["deceased"] = 0
    dfd = dfd.append(dft)
    
dfd.reset_index(inplace=True)
del dfd["index"]

In [115]:
dfwdp = pd.read_csv("../data/wiki_scrape/district_population_wiki.csv") 

dfd["population"] = np.nan
for i in range(len(dfd)):
    district_name = dfd.iloc[i]["district"]
    state_name = dfd.iloc[i]["state"]
    
    district_name_wiki = district_name
    if district_name_wiki not in ["Shahdara", "South East Delhi", "Agar Malwa", "Kra Daadi", "Lepa Rada",
            "Pakke Kessang", "Charaideo", "Mirpur", "Muzaffarabad", "Kangpokpi", "Noney", "Tengnoupal",
            "Niwari", "Other Region", "Hnahthial", "Khawzawl", "Saitual", "Airport Quarantine"]:
        if district_name=="Delhi":
            dl_population = 0
            dfdl = dfwdp[dfwdp["state"]=="Delhi"] #["population"].sum()
            for i in range(len(dfdl)):
                if ((dfdl.iloc[i]["district"]!="Shahdara") and (dfdl.iloc[i]["district"]!="South East Delhi")):
                    dl_population+=int(dfdl.iloc[i]["population"].replace(",",""))
            district_population = dl_population
        else:
            if (district_name!="Raigarh(MH)") and ((district_name!="Raigarh(CG)")) and ("(" in district_name):
                district_name_wiki = district_name.split("(")[0]
            if district_name_wiki in map_to_wiki_district_name_mapping:
                district_name_wiki = map_to_wiki_district_name_mapping[district_name]
            district_population = dfwdp[(dfwdp["district"]==district_name_wiki) & (dfwdp["state"]==state_name)]["population"].values[0]
            district_population = int(district_population.replace(",",""))

        dfd.at[(dfd["district"]==district_name) & (dfd["state"]==state_name), "population"] = district_population
dfd["cases_per_million"] = dfd["confirmed"] / dfd["population"] * 1e6
dfd.to_csv("../data/"+folder+"/derived/district_confirmed_active_recovered_deceased_population_casespm.csv", index=False)
dfd

,district,state,state_code,confirmed,active,recovered,deceased,population,cases_per_million
0,Nicobars,Andaman and Nicobar Islands,AN,0.0,0.0,0.0,0.0,36842.0,0.000000
1,North and Middle Andaman,Andaman and Nicobar Islands,AN,1.0,0.0,1.0,0.0,105597.0,9.469966
2,South Andaman,Andaman and Nicobar Islands,AN,32.0,0.0,32.0,0.0,238142.0,134.373609
3,Anantapur,Andhra Pradesh,AP,118.0,65.0,49.0,4.0,4083315.0,28.898089
4,Chittoor,Andhra Pradesh,AP,142.0,68.0,74.0,0.0,4170468.0,34.048936
5,East Godavari,Andhra Pradesh,AP,51.0,16.0,35.0,0.0,5151549.0,9.899935
6,Guntur,Andhra Pradesh,AP,399.0,166.0,225.0,8.0,4889230.0,81.607942
7,Krishna,Andhra Pradesh,AP,349.0,133.0,202.0,14.0,4529009.0,77.058800
8,Kurnool,Andhra Pradesh,AP,591.0,277.0,297.0,17.0,4046601.0,146.048498
9,Prakasam,Andhra Pradesh,AP,63.0,3.0,60.0,0.0,3392764.0,18.568931


### District wise daily: time series

In [116]:
url = "https://api.covid19india.org/districts_daily.json"
r = requests.get(url)
json_data = r.json()

In [118]:
districtwise_start_date = pd.datetime(2020, 4, 25)
dfsc = pd.read_csv("../data/extracted_data_api/state_codes.csv")

dfd = pd.DataFrame()
didx = 0
state_list = json_data["districtsDaily"].keys()
for state_name in state_list:
    print(state_name)
    # Skip "Dadra and Nagar Haveli and Daman and Diu"
    if state_name!="Dadra and Nagar Haveli and Daman and Diu":

        state_code = dfsc[dfsc["state"]==state_name]["state_code"].values[0]
        
        district_list = json_data["districtsDaily"][state_name].keys()
        for district_name in district_list:

            if district_name not in ["Unknown", "Other State", "Other States", "Other state", "Other states",
                                    "Other States*", "Other Region", "Italians*", "Italians", "Kaimur Bhabua",
                                    "Evacuees"]:
                dfcd = pd.DataFrame(json_data["districtsDaily"][state_name][district_name])

                if district_name in data_to_map_district_name_mapping:
                    district_name = data_to_map_district_name_mapping[district_name]
                if district_name in district_name_resolution:
                    district_name = district_name+"("+state_code+")"

                dfcd["district"] = district_name
                dfcd["state"] = state_name
                dfcd = dfcd[["date", "district", "state", "confirmed", "recovered", "deceased"]]
                dfcd["date"] = pd.to_datetime(dfcd["date"])
                #dfcd = dfcd[dfcd["date"]>=districtwise_start_date]
                dfcd["confirmed"] = dfcd["confirmed"].diff()
                dfcd["recovered"] = dfcd["recovered"].diff()
                dfcd["deceased"] = dfcd["deceased"].diff()
                dfcd.dropna(subset=["confirmed", "recovered", "deceased"], inplace=True)

                # Replace -ve values with 0
                dfcd.at[dfcd["confirmed"]<0, "confirmed"] = 0
                dfcd.at[dfcd["recovered"]<0, "recovered"] = 0
                dfcd.at[dfcd["deceased"]<0, "deceased"] = 0

                ################## Interpolate ##########################
                dftmp = dfcd.copy()
                df1 = dfcd.copy()
                df1["date"] = pd.to_datetime(df1["date"]).dt.date

                if len(df1[df1["confirmed"]>0])>0:
                    first_nonzero_date = df1[df1["confirmed"]>0].head(1)["date"].values[0]
                    df1 = df1[df1["date"]>first_nonzero_date]

                if len(df1)>0:
                    init_date = pd.to_datetime(df1.iloc[0]["date"]).date()
                    last_date = pd.to_datetime(df1.iloc[len(df1)-1]["date"]).date()

                    cdt = init_date
                    while (cdt < last_date):
                        if len(df1[df1["date"]==cdt])>0:
                            current_date_count = df1[df1["date"]==cdt]["confirmed"].values[0]
                            if current_date_count>0:
                                cdt+=timedelta(1)
                            elif current_date_count==0:
                                dft = df1[df1["date"]>=cdt]
                                if len(dft[dft["confirmed"]>0])>0:
                                    next_nonzero_date = pd.to_datetime(dft[dft["confirmed"]>0].head(1)["date"].values[0]).date()
                                    num_days = (next_nonzero_date - cdt).days
                                    next_nonzero_count = df1[df1["confirmed"]>0].head(1)["confirmed"].values[0]
                                    count_per_day = int(next_nonzero_count / num_days)
                                    while (cdt <= next_nonzero_date):
                                        df1.at[df1["date"]==cdt, "confirmed"] = count_per_day
                                        cdt+=timedelta(1)
                                else:
                                    cdt+=timedelta(1)
                            else:
                                print("Negative recovered")
                        else:
                            cdt+=timedelta(1)


                    cdt = init_date
                    while (cdt < last_date):
                        if len(df1[df1["date"]==cdt])>0:
                            current_date_count = df1[df1["date"]==cdt]["recovered"].values[0]
                            if current_date_count>0:
                                cdt+=timedelta(1)
                            elif current_date_count==0:
                                dft = df1[df1["date"]>=cdt]
                                if len(dft[dft["recovered"]>0])>0:
                                    next_nonzero_date = pd.to_datetime(dft[dft["recovered"]>0].head(1)["date"].values[0]).date()
                                    num_days = (next_nonzero_date - cdt).days
                                    next_nonzero_count = df1[df1["recovered"]>0].head(1)["recovered"].values[0]
                                    count_per_day = int(next_nonzero_count / num_days)
                                    while (cdt <= next_nonzero_date):
                                        df1.at[df1["date"]==cdt, "recovered"] = count_per_day
                                        cdt+=timedelta(1)
                                else:
                                    cdt+=timedelta(1)
                            else:
                                print("Negative recovered")
                        else:
                            cdt+=timedelta(1)

                    cdt = init_date
                    while (cdt < last_date):
                        if len(df1[df1["date"]==cdt])>0:
                            current_date_count = df1[df1["date"]==cdt]["deceased"].values[0]
                            if current_date_count>0:
                                cdt+=timedelta(1)
                            elif current_date_count==0:
                                dft = df1[df1["date"]>=cdt]
                                if len(dft[dft["deceased"]>0])>0:
                                    next_nonzero_date = pd.to_datetime(dft[dft["deceased"]>0].head(1)["date"].values[0]).date()
                                    num_days = (next_nonzero_date - cdt).days
                                    next_nonzero_count = df1[df1["deceased"]>0].head(1)["deceased"].values[0]
                                    count_per_day = int(next_nonzero_count / num_days)
                                    while (cdt <= next_nonzero_date):
                                        df1.at[df1["date"]==cdt, "deceased"] = count_per_day
                                        cdt+=timedelta(1)
                                else:
                                    cdt+=timedelta(1)
                            else:
                                print("Negative deceased")
                        else:
                            cdt+=timedelta(1)

                    dfcd = df1.copy()
                    ################## Interpolate end ######################


                    if didx==0:
                        dfd = dfcd.copy()
                    else:
                        dfd = dfd.append(dfcd)
                    didx+=1


dfd.to_csv("../data/"+folder+"/derived/district_daily_confirmed_recovered_deceased.csv", index=False)

Andaman and Nicobar Islands
Andhra Pradesh
Arunachal Pradesh
Assam
Bihar
Chandigarh
Chhattisgarh
Delhi
Goa
Gujarat
Haryana
Himachal Pradesh
Jammu and Kashmir
Jharkhand
Karnataka
Kerala
Ladakh
Madhya Pradesh
Maharashtra
Manipur
Meghalaya
Mizoram
Odisha
Puducherry
Punjab
Rajasthan
Tamil Nadu
Telangana
Tripura
Uttar Pradesh
Uttarakhand
West Bengal
Dadra and Nagar Haveli
Dadra and Nagar Haveli and Daman and Diu
Lakshadweep
Nagaland
Sikkim


### District growth rate  and Cases per population

In [121]:
num_days_rolling_avg = 5
init_date = pd.datetime(2020, 4, 25).date()
df = pd.read_csv("../data/"+folder+"/derived/district_daily_confirmed_recovered_deceased.csv")
df["date"] = pd.to_datetime(df["date"]).dt.date
del df["recovered"], df["deceased"]
district_list = df[df["state"]!="Delhi"]["district"].unique()
#district_list = df[df["state"]=="West Bengal"]["district"].unique()

### Set columns based on dates
columns = ["district"]
cdt = init_date
latest_date = today_date.date() - timedelta(1)
while(cdt <= latest_date):
    columns.append(pd.to_datetime(cdt.strftime("%Y-%m-%d")).date())
    cdt += timedelta(1)
dfrc = pd.DataFrame(columns=columns) 
dfcp = pd.DataFrame(columns=columns) 

for i in range(len(district_list)):
    district_name = district_list[i]
    
    dfd = df[df["district"]==district_name].copy()
    dfd = dfd[:-1]
    dfd["cumulative_count"] = dfd["confirmed"].cumsum()
    dfd["previous_day_count"] = np.nan
    l = [[0], dfd["cumulative_count"].values[:len(dfd)-1]]
    previous_day_count = [item for sublist in l for item in sublist]
    dfd["previous_day_count"] = previous_day_count
    
    ############## District growth rate ################
    dfds = dfd.copy()
    #district_name = dfd["district"].unique()[0]
    dfds["daily_growth_rate"] = (dfds["cumulative_count"] - dfds["previous_day_count"])/dfds["previous_day_count"]
    dfds["avg_growth_rate"] = dfds[["date","daily_growth_rate"]].set_index("date").rolling(num_days_rolling_avg).mean().reset_index()["daily_growth_rate"].values
    dfds = dfds[dfds["date"]>=init_date]
    dfds["avg_growth_rate"] = np.round(dfds["avg_growth_rate"] * 100, 2)
    dfds = dfds[["date","district","avg_growth_rate"]]
    dfds["date"] = pd.to_datetime(dfds["date"])
    dfds = dfds.set_index("date").resample("D").fillna("ffill").reset_index()
    dfds["date"] = pd.to_datetime(dfds["date"]).dt.date
    dfds.sort_values("date", inplace=True)
    dfds.rename(columns={"avg_growth_rate":district_name}, inplace=True)
    del dfds["district"]
    dfds = dfds.transpose()
    new_header = dfds.iloc[0] 
    dfds = dfds[1:] 
    dfds.columns = new_header
    dfds.reset_index(inplace=True)
    dfds.rename(columns={"index":"district"}, inplace=True)
    
    dfrc = dfrc.append(dfds, sort=True)
    
    
    ############## District growth rate ################
    dfdp = dfd.copy()
    if len(dfdp)>1:
        dfdp["cases_per_million"] = np.nan
        state_name = dfdp["state"].unique()[0]

        district_name_wiki = district_name
        if "(" in district_name:
            district_name_wiki = district_name.split("(")[0]
        if district_name_wiki in map_to_wiki_district_name_mapping:
            district_name_wiki = map_to_wiki_district_name_mapping[district_name]
        
        if district_name_wiki not in ["Shahdara", "South East Delhi", "Agar Malwa", "Evacuees"]:
            if len(dfwdp[(dfwdp["district"]==district_name_wiki) & (dfwdp["state"]==state_name)])==1:
                district_population = dfwdp[(dfwdp["district"]==district_name_wiki) & (dfwdp["state"]==state_name)]["population"].values[0]
                district_population = int(district_population.replace(",",""))
                dfdp["cases_per_million"] = dfdp["confirmed"]/district_population*1e6
            elif len(dfwdp[dfwdp["district"]==district_name_wiki])==0:
                print(district_name + " population data not found")
            elif len(dfwdp[dfwdp["district"]==district_name_wiki])>1:
                print(district_name + " more than one population data found")

        dfdp = dfdp[["date","cases_per_million"]]
        dfdp = dfdp[dfdp["date"]>=init_date]
        dfdp["cases_per_million"] = np.round(dfdp["cases_per_million"],2)
        dfdp.rename(columns={"cases_per_million":district_name}, inplace=True)
        dfdp = dfdp.transpose()
        new_header = dfdp.iloc[0] 
        dfdp = dfdp[1:] 
        dfdp.columns = new_header
        dfdp.reset_index(inplace=True)
        dfdp.rename(columns={"index":"district"}, inplace=True)

        dfcp = dfcp.append(dfdp, sort=True)
    else:
        dfm = pd.DataFrame()
        dfm.at[0, "district"] = district_name
        dfcp = dfcp.append(dfm, sort=True)

dfd = pd.read_csv("../data/"+folder+"/derived/district_confirmed_active_recovered_deceased_population_casespm.csv")
dfd = dfd[["district", "state_code", "confirmed"]]
dfdrc = dfd.set_index("district").join(dfrc.set_index("district")).reset_index()
dfdrc["district"] = dfdrc["district"] + ", " + dfdrc["state_code"]
del dfdrc["state_code"]
dfdrc.rename(columns={"confirmed":"total_case_count"}, inplace=True)
dfdrc.to_csv("../data/"+folder+"/code_data/districtwise_growth_rate.csv", index=False)
dfdrc

dfd = pd.read_csv("../data/"+folder+"/derived/district_confirmed_active_recovered_deceased_population_casespm.csv")
dfd = dfd[["district", "state_code", "confirmed"]]
dfdcp = dfd.set_index("district").join(dfcp.set_index("district")).reset_index()
dfdcp["district"] = dfdcp["district"] + ", " + dfdcp["state_code"]
del dfdcp["state_code"]
dfdcp.rename(columns={"confirmed":"total_case_count"}, inplace=True)
dfdcp.to_csv("../data/"+folder+"/code_data/districtwise_latest_cases_per_population.csv", index=False)


### District case growth rate and cases per population: derived from raw data

In [173]:
### Derive districtwise time series from raw data

df = pd.read_csv("../data/09052020/one_time/full_data_patient_level_withDistrictMapped_withStatusMapped.csv")
df["date"] = pd.to_datetime(df["date"])
df.sort_values("date")
df = df[["date", "district", "state"]]
df["count"] = 1
### Combine all Delhi cases under one district with name "Delhi"
df.at[df["state"]=="Delhi", "district"] = "Delhi"
dfts = df.groupby(["state", "district", "date"]).count().reset_index()
dfts

### District and state list
dfd = pd.read_csv("../data/extracted_data/district_list.csv")[["district", "state"]]
dfd.at[dfd["state"]=="Delhi", "district"] = "Delhi"
state_list = dfd["state"].unique()
#state_list = ["Karnataka"]

### Get time series of cases for each district
start_date = pd.to_datetime(df.head(1)["date"].values[0])
end_date = pd.to_datetime(df.tail(1)["date"].values[0])

dfdd = pd.DataFrame()
idx = 0
dc = 0
for sid in range(len(state_list)):
    state_name = state_list[sid]
    print(state_name)
    
    state_district_list = dfd[dfd["state"]==state_name]["district"].unique()
    #state_district_list = ["Mysuru"]
    dc += len(state_district_list)
    for did in range(len(state_district_list)):
        district_name = state_district_list[did]
        
        d = start_date
        while (d <= end_date):
            dfdd.at[idx, "state"] = state_name
            dfdd.at[idx, "district"] = district_name
            dfdd.at[idx, "date"] = d
            
            dfc = dfts[(dfts["state"]==state_name) & (dfts["district"]==district_name) & (dfts["date"]==d)]
            if len(dfc)==0:
                dfdd.at[idx, "count"] = 0
            elif len(dfc)==1:
                dfdd.at[idx, "count"] = dfc["count"].values[0]
            else:
                print("Error: " + state_name + " " + district_name + " " + date)
            
            idx += 1
            d += timedelta(days=1)
#dfdd.to_csv("../data/"+folder+"/derived/districtwise_time_series.csv", index=False)

Andaman and Nicobar Islands
Andhra Pradesh
Arunachal Pradesh
Assam
Bihar
Chandigarh
Chhattisgarh
Dadra and Nagar Haveli
Daman And Diu
Delhi
Goa
Gujarat
Haryana
Himachal Pradesh
Jammu and Kashmir
Jharkhand
Karnataka
Kerala
Ladakh
Lakshadweep
Madhya Pradesh
Maharashtra
Manipur
Meghalaya
Mizoram
Nagaland
Odisha
Puducherry
Punjab
Rajasthan
Sikkim
Tamil Nadu
Telangana
Tripura
Uttar Pradesh
Uttarakhand
West Bengal


In [177]:
max_growth_rate_nan_cutoff = 150
init_date = pd.datetime(2020, 3, 15)
num_days_rolling_avg = 7

dfdd = pd.read_csv("../data/09052020/derived/districtwise_time_series.csv")
dfdd["date"] = pd.to_datetime(dfdd["date"])
state_list = dfdd["state"].unique()
district_list = dfdd["district"].unique()

latest_date = pd.to_datetime(dfdd.sort_values("date").tail(1)["date"].values[0])
two_weeks_past_date = latest_date - timedelta(days=14)

#dfdc = dfdd.groupby(["state", "district"]).sum()["count"].reset_index()
#dfdc = dfdc[dfdc["count"]>min_case_count]
#district_list_with_min_cases = dfdc["district"].unique()

# State code
dfsc = pd.read_csv("../data/extracted_data/state_codes.csv")

# District population
dfwdp = pd.read_csv("../data/wiki_scrape/district_population_wiki_withNameMapped.csv")

ridx = 0
cidx = 0
cpidx = 0
dfrc = pd.DataFrame() ## Current growth rate for all districts ##with min_case_count
dfcc = pd.DataFrame() ## Current Growth rate for all districts
dfrg = pd.DataFrame() ## Latest two weeks Growth rate for all districts
dfcp = pd.DataFrame() ## Latest two weeks cases/population for all districts

#state_list = ["Rajasthan"]
for sid in range(len(state_list)):
    state_name = state_list[sid]
    if state_name=="Daman And Diu":
        state_name = state_name.replace("And","and")
    state_code = dfsc[dfsc["state"]==state_name]["state_code"].values[0]
    print("==== " + state_name + " ====")
    
    state_district_list = dfdd[dfdd["state"]==state_name]["district"].unique()
    # Remove nan names
    state_district_list = [l for l in state_district_list if not pd.isnull(l)]    
    # Handle special case of Delhi (covid19.org raw data has no district wise data for Delhi)
    # Combine all districts of Delhi to one entity
    if state_name=="Delhi":
        state_district_list = ["Delhi"]
    
    #print(state_district_list)
    #state_district_list = ["Jaisalmer"]
    for did in range(len(state_district_list)):
        district_name = state_district_list[did]
        
        if district_name in district_name_mapping.keys():
            district_name_mapped = district_name_mapping[district_name]
            dfdd['district'] = dfdd['district'].replace({district_name:district_name_mapped})
            district_name = district_name_mapped

        if district_name in district_name_resolution:
            district_name_resolved = resolve_district_name(state_name, district_name)
            dfdd['district'] = dfdd['district'].replace({district_name:district_name_resolved})
            district_name = district_name_resolved

        district_population = np.nan
        if district_name=="Delhi":
            district_population = dfwdp[dfwdp["state"]=="Delhi"]["population"].sum()
        else:
            if len(dfwdp[dfwdp["district"]==district_name])==0:
                print(district_name + " population data not found")
            if len(dfwdp[dfwdp["district"]==district_name])>1:
                print(district_name + " more than one population data found")
            if len(dfwdp[dfwdp["district"]==district_name])==1:
                district_population = dfwdp[dfwdp["district"]==district_name]["population"].values[0]
        
        if not pd.isnull(district_name):
            #print(district_name)
            dfd = dfdd[(dfdd["state"]==state_name) & (dfdd["district"]==district_name)].copy()
            #d = start_date
            #while (d <= end_date):

            dfd["cumulative_count"] = dfd["count"].cumsum()
            dfd["previous_day_count"] = np.nan

            l = [[0], dfd["cumulative_count"].values[:len(dfd)-1]]
            previous_day_count = [item for sublist in l for item in sublist]

            # District total case count
            district_total_case_count = dfd[dfd["date"]==latest_date]["cumulative_count"].values[0]

            ######### Current Growth rate and latest two weeks growth for all districts ########
            ### ------ For districit hotspot table----- ###
            dfd["previous_day_count"] = previous_day_count
            #dfds = dfd[dfd["previous_day_count"]!=0].copy()
            dfds = dfd.copy()
            dfds["daily_growth_rate"] = (dfds["cumulative_count"] - dfds["previous_day_count"])/dfds["previous_day_count"]
            dfds["avg_growth_rate"] = dfds[["date","daily_growth_rate"]].set_index("date").rolling(num_days_rolling_avg).mean().reset_index()["daily_growth_rate"].values
            dfds = dfds[dfds["date"]>=init_date]
            dfds["avg_growth_rate"] = np.round(dfds["avg_growth_rate"] * 100, 2)
            ## Smooth curve: 
            dfds.at[dfds["avg_growth_rate"]>=max_growth_rate_nan_cutoff, "avg_growth_rate"] = np.nan
            if len(dfds)==0:
                dfc = dfd[dfd["date"]==latest_date][["district", "state", "cumulative_count"]].copy()
                dfc["avg_growth_rate"] = np.nan
                dfc.rename(columns={"cumulative_count":"total_cases", "avg_growth_rate":"current_avg_growth_rate"}, inplace=True)
                cdt = two_weeks_past_date
                gh=0;
                while (cdt <= latest_date):
                    dfc.at[dfc.index, "day"+str(gh)] = np.nan
                    cdt += timedelta(days=1)
                    gh+=1
            else:
                dfc = dfds[dfds["date"]==latest_date][["district", "state", "cumulative_count", "avg_growth_rate"]].copy()
                #dfc.at[dfc["district"]==district_name, "district"] = district_name+", "+state_code
                dfc.rename(columns={"cumulative_count":"total_cases", "avg_growth_rate":"current_avg_growth_rate"}, inplace=True)
                cdt = two_weeks_past_date
                gh=0;
                while (cdt <= latest_date):
                    dfc.at[dfc.index, "day"+str(gh)] = dfds[dfds["date"]==cdt]["avg_growth_rate"].values[0]
                    cdt += timedelta(days=1)
                    gh+=1
            if cidx==0:
                dfcc = dfc.copy()
            else:
                dfcc = dfcc.append(dfc)
            cidx += 1

            ########## Growth rate for all days for all districts ###########
            ### ------ For districit growth rate curve----- ###
            dfr = dfds[["date", "avg_growth_rate"]].copy()
            #dfr = dfr[dfr["date"]>two_weeks_past_date]
            dfr["date"] = pd.to_datetime(dfr["date"]).dt.date
            dfr.rename(columns={"date":"district","avg_growth_rate":district_name+", "+state_code}, inplace=True)
            dfr.reset_index(inplace=True)
            del dfr["index"]
            dfr.at[-1, "district"] = "total_case_count"
            dfr.at[-1, district_name+", "+state_code] = district_total_case_count
            dfr.sort_index(inplace=True)
            dfr = dfr.transpose()
            new_header = dfr.iloc[0] #grab the first row for the header
            dfr = dfr[1:] #take the data less the header row
            dfr.columns = new_header
            #if district_name in district_list_with_min_cases:
            if ridx==0:
                dfrc = dfr.copy()
            else:
                dfrc = dfrc.append(dfr)
            ridx += 1
            
            """
            ######## Latest two weeks Growth rate for all districts ########
            dfrgd = dfds[["date", "avg_growth_rate"]].copy()
            dfrgd = dfrgd[dfrgd["date"]>two_weeks_past_date]
            dfrgd["date"] = pd.to_datetime(dfrgd["date"]).dt.date
            dfrgd.rename(columns={"date":"district","avg_growth_rate":district_name+", "+state_code}, inplace=True)
            dfrgd.reset_index(inplace=True)
            del dfrgd["index"]
            dfrgd.at[-1, "district"] = "total_case_count"
            dfrgd.at[-1, district_name+", "+state_code] = district_total_case_count
            dfrgd.sort_index(inplace=True)
            dfrgd = dfrgd.transpose()
            new_header = dfrgd.iloc[0] #grab the first row for the header
            dfrgd = dfrgd[1:] #take the data less the header row
            dfrgd.columns = new_header
            #dfrgd = dfrr[[col for col in dfrr.columns if col>two_weeks_past_date]].copy()
            if cidx==0:
                dfrg = dfrgd.copy()
            else:
                dfrg = dfrg.append(dfrgd)
            """

            ######## Latest two weeks cases per population for all districts ##############
            ### ------ For districit case density curve----- ###
            #if district_total_case_count>=min_case_count_to_compute_case_density:
            dfcpd = dfds[["district", "date", "count"]].copy()
            dfcpd["avg_daily_cases"] = dfds[["date","count"]].set_index("date").rolling(5).mean().reset_index()["count"].values
            dfcpd["population"] = district_population
            dfcpd["cases_per_lakh"] = dfcpd["avg_daily_cases"] / dfcpd["population"] * 100000
            dfcpd = dfcpd[["date", "cases_per_lakh"]].copy()
            dfcpd.rename(columns={"cases_per_lakh":district_name+", "+state_code}, inplace=True)
            dfcpd = dfcpd[dfcpd["date"]>two_weeks_past_date]
            dfcpd["date"] = pd.to_datetime(dfcpd["date"]).dt.date
            dfcpd.reset_index(inplace=True)
            del dfcpd["index"]
            dfcpd.at[-1, "date"] = "total_case_count"
            dfcpd.at[-1, district_name+", "+state_code] = district_total_case_count
            dfcpd.sort_index(inplace=True)

            dfcpd = dfcpd.transpose()
            new_header = dfcpd.iloc[0] #grab the first row for the header
            dfcpd = dfcpd[1:] #take the data less the header row
            dfcpd.columns = new_header
            #dfcpd = dfcpd[[col for col in dfcpd.columns if col>two_weeks_past_date]].copy()
            if cpidx==0:
                dfcp = dfcpd.copy()
            else:
                dfcp = dfcp.append(dfcpd)
            cpidx+=1
                       

dfcp.to_csv("../data/09052020/code_data/districtwise_latest_cases_per_population_derivedFromRawData.csv")
##dfcc.to_csv("../data/extracted_data_api/code_data/districtwise_current_growth_rate_all_withDistrictNameMapped.csv", index=False)
dfrc.to_csv("../data/09052020/code_data/districtwise_growth_rate_derivedFromRawData.csv")


==== Andaman and Nicobar Islands ====
==== Andhra Pradesh ====
==== Arunachal Pradesh ====
Pakke Kessang population data not found
==== Assam ====
Biswanath population data not found
South Salmara Mancachar population data not found
==== Bihar ====
==== Chandigarh ====
==== Chhattisgarh ====
==== Dadra and Nagar Haveli ====
==== Daman and Diu ====
==== Delhi ====
==== Goa ====
==== Gujarat ====
==== Haryana ====
==== Himachal Pradesh ====
==== Jammu and Kashmir ====
Mirpur population data not found
Muzaffarabad population data not found
==== Jharkhand ====
==== Karnataka ====
==== Kerala ====
==== Ladakh ====
==== Lakshadweep ====
==== Madhya Pradesh ====
==== Maharashtra ====
==== Manipur ====
==== Meghalaya ====
==== Mizoram ====
==== Nagaland ====
==== Odisha ====
==== Puducherry ====
==== Punjab ====
==== Rajasthan ====
==== Sikkim ====
==== Tamil Nadu ====
Tirupathur population data not found
==== Telangana ====
==== Tripura ====
==== Uttar Pradesh ====
==== Uttarakhand ====
==== 

### Basic maps (district, statewise - cases, deaths)

#### State map data

In [128]:
df = pd.read_csv('../data/'+folder+'/state_wise_current.csv')

with open("../data/india_geojson_raw/india_topojson.json") as topodata:
    raw_data = json.load(topodata)

data = raw_data
state_list = data["objects"]["india"]["geometries"]

for i in range(len(state_list)):
    state_name = state_list[i]["properties"]["st_nm"]
    if (state_name=="Dadra and Nagar Haveli"):
        state_name = "Dadra and Nagar Haveli and Daman and Diu"

    df_state = df[df["State"]==state_name].copy()

    if len(df_state)==1:
        data["objects"]["india"]["geometries"][i]["properties"]["confirmed"] = str(df_state["Confirmed"].values[0])
        data["objects"]["india"]["geometries"][i]["properties"]["active"] = str(df_state["Active"].values[0])
        data["objects"]["india"]["geometries"][i]["properties"]["deaths"] = str(df_state["Deaths"].values[0])
        data["objects"]["india"]["geometries"][i]["properties"]["recovered"] = str(df_state["Recovered"].values[0])
        
    elif len(df_state)==0:
        print(state_name + " data not found")
    elif len(df_state)>1:
        print(state_name + " more than one data found")
    else:
        print(state_name + " other error")

#with open('../data/india_geojson_data_api/india.json', 'w') as outfile:
with open('../data/'+folder+'/code_data/india.json', 'w') as outfile:
    json.dump(data, outfile)

Hello data not found


#### District map data

In [129]:
### Get data: cases, deaths, cases_per_million
df = pd.read_csv("../data/"+folder+"/derived/district_confirmed_active_recovered_deceased_population_casespm.csv")
### Get data: district wise growth rate
dfcg = pd.read_csv("../data/"+folder+"/code_data/districtwise_growth_rate.csv")
dfcg = dfcg[["district", dfcg.columns[-1]]]
dfcg.rename(columns={dfcg.columns[-1]:"current_case_growth_rate"}, inplace=True)


dfmd = pd.read_csv("../data/district_list_map.csv")
delhi_district_list = dfmd[dfmd["state"]=="Delhi"]["district"].values

state_name_list = []

for fp in os.listdir("../data/state_geojson_raw/"):
    if not fp.startswith(".") and fp.endswith(".json"):
        state_name = fp.split(".")[0]
        state_name_mapped = state_name_mapping[state_name]
        state_name_list.append(state_name)
        print(state_name)

        with open("../data/state_geojson_raw/"+fp) as topodata:
            raw_data = json.load(topodata)

        data = raw_data
        district_list = data["objects"][state_name+"_district"]["geometries"]
        
        if state_name=="tamilnadu":
            data["transform"]["scale"] = [0.0006973698971092601, 0.0006462543480830687]
            data["transform"]["translate"] = [76.20310065122152, 8.107847033856163]
        
        for i in range(len(district_list)):
            district_name = data["objects"][state_name+"_district"]["geometries"][i]["properties"]["district"]
            
            # Combine districts of Delhi
            if district_name in delhi_district_list:
                district_name = "Delhi"
                data["objects"][state_name+"_district"]["geometries"][i]["properties"]["district"] = "Delhi"

            ## District name not found
            if (0): #district_name in district_name_unmapped:
                data["objects"][state_name+"_district"]["geometries"][i]["properties"]["cases"] = str(-1)
                data["objects"][state_name+"_district"]["geometries"][i]["properties"]["deaths"] = str(-1)
                #data["objects"][state_name+"_district"]["geometries"][i]["properties"]["current_case_growth_rate"] = str(-1)
                #data["objects"][state_name+"_district"]["geometries"][i]["properties"]["case_density"] = str(-1)
            else:
                if ((district_name=="Raigarh") and (state_name_mapped=="Maharashtra")):
                    district_name = "Raigarh(MH)"
                if ((district_name=="Raigarh") and (state_name_mapped=="Chhattisgarh")):
                    district_name = "Raigarh(CG)"
                
                df_district = df[(df["district"]==district_name) & (df["state"]==state_name_mapped)].copy()
                state_code = df[df["state"]==state_name_mapped]["state_code"].unique()[0]
                dfcg_district_name = district_name + ", " + state_code
                dfcg_district = dfcg[dfcg["district"]==dfcg_district_name]
                
                if len(df_district)==1:
                    data["objects"][state_name+"_district"]["geometries"][i]["properties"]["cases"] = str(df_district["confirmed"].values[0])
                    data["objects"][state_name+"_district"]["geometries"][i]["properties"]["deaths"] = str(df_district["deceased"].values[0])
                    data["objects"][state_name+"_district"]["geometries"][i]["properties"]["case_density"] = str(df_district["cases_per_million"].values[0])
                elif len(df_district)==0:
                    print(district_name + " data not found")
                elif len(df_district)>1:
                    print(district_name + " more than one data found")
                else:
                    print(district_name + " other error")
                    
                if len(dfcg_district)==1:
                    data["objects"][state_name+"_district"]["geometries"][i]["properties"]["current_case_growth_rate"] = str(dfcg_district["current_case_growth_rate"].values[0])
                elif len(dfcg_district)==0:
                    print(district_name + " cgr data not found")
                elif len(dfcg_district)>1:
                    print(district_name + " cgr more than one data found")
                else:
                    print(district_name + " cgr other error")
                    
        with open('../data/'+folder+'/code_data/district_data_map/'+state_name+'.json', 'w') as outfile:
            json.dump(data, outfile)


odisha
telangana
meghalaya
karnataka
haryana
bihar
andhrapradesh
jammukashmir
westbengal
kerala
chhattisgarh
andamannicobarislands
jharkhand
ladakh
uttarpradesh
mizoram
lakshadweep
nagaland
tamilnadu
dadranagarhaveli
delhi
puducherry
madhyapradesh
arunachalpradesh
uttarakhand
manipur
tripura
gujarat
goa
assam
maharashtra
punjab
sikkim
rajasthan
chandigarh
himachalpradesh


### Daily and overall cases, deaths

In [130]:
df = pd.read_csv('../data/'+folder+'/case_time_series.csv')
df.rename(columns={"Date":"date", "Daily Confirmed": "daily_confirmed", "Total Confirmed": "total_confirmed",
                   "Daily Recovered":"daily_recovered", "Total Recovered":"total_recovered",
                   "Daily Deceased":"daily_deaths", "Total Deceased":"total_deaths"}, inplace=True)
df["date"] = df["date"].str.strip()
for i in range(len(df)):
    df.at[i, "date"] = df.iloc[i]["date"] + " 2020"
df["date"] = pd.to_datetime(df["date"])
#df.columns = ["daily_confirmed", "daily_deaths", "daily_recovered", "date", "total_confirmed", "total_deaths", "total_recovered"]
df = df[["date", "daily_confirmed", "total_confirmed", "daily_deaths", "total_deaths", "daily_recovered", "total_recovered"]] 

dfr = df.set_index("date").rolling(7).mean().reset_index()[["date", "daily_confirmed", "daily_deaths"]]
dfr.columns = ["date", "rolling_avg_confirmed", "rolling_avg_deaths"]
df = df.set_index("date").join(dfr.set_index("date")).reset_index()

df.to_csv("../data/"+folder+"/code_data/overall_and_daily_cases_deaths.csv", index=False)
df.head(2)

,date,daily_confirmed,total_confirmed,daily_deaths,total_deaths,daily_recovered,total_recovered,rolling_avg_confirmed,rolling_avg_deaths
0,2020-01-30,1,1,0,0,0,0,NaN,NaN
1,2020-01-31,0,1,0,0,0,0,NaN,NaN


### State - Cases, deaths, testspm, ntod -- For state testing vs growth animation

In [131]:
# Get case count
dfsc = pd.read_csv("../data/"+folder+"/statewise_daily_confirmed.csv")
del dfsc["TT"]
dfsc["date"] = pd.to_datetime(dfsc["date"]).dt.date
dfsc["date"] = dfsc["date"].apply(lambda x: x.strftime('%Y-%m-%d'))
dfsc = dfsc.set_index("date").cumsum().reset_index()
dfsc = dfsc.transpose()
new_header = dfsc.iloc[0]
dfsc = dfsc[1:]
dfsc.columns = new_header
dfsc.columns = [col+"_cases" for col in dfsc.columns]
dfsc.reset_index(inplace=True)
dfsc.rename(columns={"index":"state"}, inplace=True)


# Get death count
dfsd = pd.read_csv("../data/"+folder+"/statewise_daily_deceased.csv")
del dfsd["TT"] #, dfsd["Unnamed: 39"]
dfsd["date"] = pd.to_datetime(dfsd["date"]).dt.date
dfsd["date"] = dfsd["date"].apply(lambda x: x.strftime('%Y-%m-%d'))
dfsd = dfsd.set_index("date").cumsum().reset_index()
dfsd = dfsd.transpose()
new_header = dfsd.iloc[0]
dfsd = dfsd[1:]
dfsd.columns = new_header
dfsd.columns = [col+"_deaths" for col in dfsd.columns]
dfsd.reset_index(inplace=True)
dfsd.rename(columns={"index":"state"}, inplace=True)


#### Join: Cases and deaths
dfs = dfsc.set_index("state").join(dfsd.set_index("state")).reset_index()


# Get tests per million
dfst = pd.read_csv("../data/"+folder+"/statewise_testing.csv")[["State", "Updated On", "Total Tested"]]
dfst.rename(columns={"State":"state_name", "Total Tested":"tests", "Updated On":"date"}, inplace=True)

# Get state population
dfsp = pd.read_csv("../data/wiki_scrape/district_population_wiki_withNameMapped.csv")
dfsp = dfsp.groupby(["state", "state_code"]).sum()["population"].reset_index()
dfsp.rename(columns={"state":"state_name"}, inplace=True)

# Join state population and state testing data
dfst = dfst.set_index("state_name").join(dfsp.set_index("state_name"), how="inner").reset_index()
dfst["date"] = pd.to_datetime(dfst["date"], dayfirst=True).dt.date

# Tests per million population
dfst["testspm"] = dfst["tests"]/dfst["population"] * 1e6
dfst["testspm"] = np.round(dfst["testspm"], 2)
del dfst["tests"], dfst["population"]


dfsc = pd.read_csv("../data/"+folder+"/statewise_daily_confirmed.csv")
dfsc["date"] = pd.to_datetime(dfsc["date"], dayfirst=True).dt.date
start_date = dfsc["date"][0]
end_date = dfsc["date"][len(dfsc)-1]

dfit = pd.DataFrame()
state_list = dfst["state_name"].unique()

for i in range(len(state_list)):
    state_name = state_list[i]
    dft = dfst[dfst["state_name"]==state_name].copy()
    dft.dropna(subset=["testspm"], inplace=True)
    dft.drop_duplicates(inplace=True, subset=["date"], keep="last")
    state_code = dft["state_code"].unique()[0]
    dfi = pd.DataFrame()
    cdt = start_date
    idx=0
    while cdt<=end_date:
        dfi.at[idx, "date"] = cdt
        cdt += timedelta(days=1)
        idx +=1
    dfi = dfi.set_index("date").join(dft.set_index("date")).reset_index()
    dfi["state_name"] = dfi["state_name"].fillna(state_name)
    dfi["state_code"] = dfi["state_code"].fillna(state_code)
    dfi["testspm"] = dfi["testspm"].fillna(method="ffill")
    dfi["testspm"] = dfi["testspm"].fillna(0)

    dfi["date"] = dfi["date"].apply(lambda x: x.strftime('%Y-%m-%d')+"_testspm")
    del dfi["state_name"], dfi["state_code"]
    dfi.rename(columns={"testspm":state_code}, inplace=True)
    dfi = dfi.transpose()
    new_header = dfi.iloc[0]
    dfi = dfi[1:]
    dfi.columns = new_header
    dfi.reset_index(inplace=True)
    dfi.rename(columns={"index":"state"}, inplace=True)
    
    if i==0:
        dfit = dfi.copy()
    else:
        dfit = dfit.append(dfi) #, sort=True)


        
#### Join: tests
dfs = dfs.set_index("state").join(dfit.set_index("state")).reset_index()

        
        
        
## Get growth rate
min_case_count_to_compute_doubling_rate = 50
dfsc = pd.read_csv("../data/"+folder+"/statewise_daily_confirmed.csv")
del dfsc["TT"] #, dfsc["Unnamed: 39"]
dfsc["date"] = pd.to_datetime(dfsc["date"]).dt.date
dfsc["date"] = dfsc["date"].apply(lambda x: x.strftime('%Y-%m-%d'))
dfsc = dfsc.set_index("date").cumsum().reset_index()

dfsg = pd.DataFrame()
stidx = 0
for col in dfsc.columns:
    if col not in ["date"]:
        dfss = dfsc[["date", col]].copy()
        dfss["prev"] = np.nan
        l = [[0], dfss[col].values[:len(dfss)-1]]
        previous_day_count = [item for sublist in l for item in sublist]
        dfss["prev"] = previous_day_count
        
        ### Days to double
        dfss["ndays_to_double"] = 1/np.log2(dfss[col]/dfss["prev"])
        dfss["ndays_to_double_ravg"]  = dfss.set_index("date").rolling(5).mean()["ndays_to_double"].values        
        dfss.at[dfss[col]<min_case_count_to_compute_doubling_rate, "ndays_to_double_ravg"] = np.nan
        
        dfss["ndays_to_double_ravg"] = np.round(dfss["ndays_to_double_ravg"], 2)
        del dfss[col], dfss["prev"], dfss["ndays_to_double"]
        dfss.rename(columns={"ndays_to_double_ravg":col}, inplace=True)
        dfss["date"] = pd.to_datetime(dfss["date"]).dt.date
        dfss["date"] = dfss["date"].apply(lambda x: x.strftime('%Y-%m-%d')+"_ntod")
        dfss = dfss.transpose()
        new_header = dfss.iloc[0]
        dfss = dfss[1:]
        dfss.columns = new_header
        dfss.reset_index(inplace=True)
        dfss.rename(columns={"index":"state"}, inplace=True)
        
        if stidx==0:
            dfsg = dfss.copy()
        else:
            dfsg = dfsg.append(dfss)
        stidx+=1

        
#### Join: growth rate
dfs = dfs.set_index("state").join(dfsg.set_index("state")).reset_index()

dfsp = pd.read_csv("../data/wiki_scrape/district_population_wiki_withNameMapped.csv")
dfsp = dfsp.groupby(["state", "state_code"]).sum()["population"].reset_index()
dfsp.rename(columns={"state":"state_name", "state_code":"state"}, inplace=True)

dfss = dfs.set_index("state").join(dfsp[["state","state_name"]].set_index("state"), how="inner").reset_index()
l = [["state"], ["state_name"], dfss.columns[1:len(dfss.columns)-1]]
columns = [item for sublist in l for item in sublist]
dfss = dfss[columns]
dfss.to_csv("../data/"+folder+"/code_data/state_case_death_testpm_ntod.csv", index=False) 
dfss

,state,state_name,2020-03-14_cases,2020-03-15_cases,2020-03-16_cases,2020-03-17_cases,2020-03-18_cases,2020-03-19_cases,2020-03-20_cases,2020-03-21_cases,2020-03-22_cases,2020-03-23_cases,2020-03-24_cases,2020-03-25_cases,2020-03-26_cases,2020-03-27_cases,2020-03-28_cases,2020-03-29_cases,2020-03-30_cases,2020-03-31_cases,2020-04-01_cases,2020-04-02_cases,2020-04-03_cases,2020-04-04_cases,2020-04-05_cases,2020-04-06_cases,2020-04-07_cases,2020-04-08_cases,2020-04-09_cases,2020-04-10_cases,2020-04-11_cases,2020-04-12_cases,2020-04-13_cases,2020-04-14_cases,2020-04-15_cases,2020-04-16_cases,2020-04-17_cases,2020-04-18_cases,2020-04-19_cases,2020-04-20_cases,2020-04-21_cases,2020-04-22_cases,2020-04-23_cases,2020-04-24_cases,2020-04-25_cases,2020-04-26_cases,2020-04-27_cases,2020-04-28_cases,2020-04-29_cases,2020-04-30_cases,...,2020-03-25_ntod,2020-03-26_ntod,2020-03-27_ntod,2020-03-28_ntod,2020-03-29_ntod,2020-03-30_ntod,2020-03-31_ntod,2020-04-01_ntod,2020-04-02_ntod,2020-04-03_ntod,2020-04-04_ntod,2020-04-05_ntod,2020-04-06_ntod,2020-04-07_ntod,2020-04-08_ntod,2020-04-09_ntod,2020-04-10_ntod,2020-04-11_ntod,2020-04-12_ntod,2020-04-13_ntod,2020-04-14_ntod,2020-04-15_ntod,2020-04-16_ntod,2020-04-17_ntod,2020-04-18_ntod,2020-04-19_ntod,2020-04-20_ntod,2020-04-21_ntod,2020-04-22_ntod,2020-04-23_ntod,2020-04-24_ntod,2020-04-25_ntod,2020-04-26_ntod,2020-04-27_ntod,2020-04-28_ntod,2020-04-29_ntod,2020-04-30_ntod,2020-05-01_ntod,2020-05-02_ntod,2020-05-03_ntod,2020-05-04_ntod,2020-05-05_ntod,2020-05-06_ntod,2020-05-07_ntod,2020-05-08_ntod,2020-05-09_ntod,2020-05-10_ntod,2020-05-11_ntod,2020-05-12_ntod,2020-05-13_ntod
0,AN,Andaman and Nicobar Islands,0,0,0,0,0,0,0,0,0,0,0,0,1,6,9,9,10,10,10,10,10,10,10,10,10,11,11,11,11,11,11,11,11,11,12,14,15,16,17,18,22,29,33,33,33,33,33,33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AP,Andhra Pradesh,1,1,1,1,1,3,3,5,6,7,8,10,11,13,19,21,23,44,111,149,164,192,256,303,314,348,363,381,405,420,439,484,525,534,572,603,647,722,757,813,893,955,1016,1097,1177,1259,1332,1403,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.64,3.74,3.8,3.16,3.43,4.1,7.52,7.42,9.83,12.21,13.65,13.58,15.36,13.5,12.34,18.23,16.43,15.92,16.47,16.03,10.8,10.73,9.58,9.68,10.65,9.53,9.56,10.14,10.53,10.96,12.47,13.84,15.49,16.38,17.19,17.92,19.04,20.11,22.91,24.85,28.11,27.65,28.98
2,AR,Arunachal Pradesh,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AS,Assam,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,16,16,23,26,26,26,28,28,29,29,29,30,32,33,33,35,35,35,35,35,35,35,36,36,36,36,36,38,38,43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.76,21.38,19.51,NaN,NaN
4,BR,Bihar,0,0,0,0,0,0,0,0,2,3,3,4,7,9,11,15,15,21,24,29,31,32,32,32,38,39,58,60,64,64,66,66,72,83,85,86,96,113,126,143,170,223,251,277,346,366,403,425,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.5,20.76,21.06,16.33,5.28,4.53,4.85,4.99,4.51,6.18,7.11,8.54,8.64,12.4,11.85,16.99,24.91,34.07,39.16,39.93,35.93,26.35,18.09,9.49,8.51
5,CH,Chandigarh,0,0,0,0,0,1,5,5,6,7,7,7,7,8,8,8,13,15,17,18,18,18,18,18,18,18,18,19,19,21,21,21,21,21,21,23,26,26,27,27,27,27,28,36,45,56,68,74,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.33,4.16,4.41,5.89,9.67,11.71,10.97,13.54,13.07,10.43,8.62,13.64,13.34,15.96,20.74
6,DD,Dadra and Nagar Haveli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

### State case count

In [132]:
df = pd.read_csv("../data/"+folder+"/statewise_daily_confirmed.csv")
del df["TT"]
df = df.set_index("date").cumsum().reset_index()
df["date"] = pd.to_datetime(df["date"]).dt.date
dfsc = pd.read_csv("../data/extracted_data_api/state_codes.csv")
df = df.transpose()
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header
df.reset_index(inplace=True)
df.rename(columns={"index":"state_code"}, inplace=True)
df = dfsc.set_index("state_code").join(df.set_index("state_code")).reset_index()
df.rename(columns={"state":"state_name"}, inplace=True)
del df["state_code"]

dfntod = pd.read_csv("../data/"+folder+"/code_data/state_case_death_testpm_ntod.csv")
dfntod = dfntod[["state_name", dfntod.columns[-1]]]
dfntod.rename(columns={dfntod.columns[-1]:"latest_ntod"}, inplace=True)

df = dfntod.set_index("state_name").join(df.set_index("state_name")).reset_index()
df.to_csv("../data/"+folder+"/code_data/state_case_count.csv", index=False)
df

,state_name,latest_ntod,2020-03-14,2020-03-15,2020-03-16,2020-03-17,2020-03-18,2020-03-19,2020-03-20,2020-03-21,2020-03-22,2020-03-23,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04,2020-05-05,2020-05-06,2020-05-07,2020-05-08,2020-05-09,2020-05-10,2020-05-11,2020-05-12,2020-05-13
0,Andaman and Nicobar Islands,NaN,0,0,0,0,0,0,0,0,0,0,0,0,1,6,9,9,10,10,10,10,10,10,10,10,10,11,11,11,11,11,11,11,11,11,12,14,15,16,17,18,22,29,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33
1,Andhra Pradesh,28.98,1,1,1,1,1,3,3,5,6,7,8,10,11,13,19,21,23,44,111,149,164,192,256,303,314,348,363,381,405,420,439,484,525,534,572,603,647,722,757,813,893,955,1016,1097,1177,1259,1332,1403,1463,1525,1583,1650,1717,1777,1833,1887,1930,1980,2018,2089,2137
2,Arunachal Pradesh,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,Assam,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,16,16,23,26,26,26,28,28,29,29,29,30,32,33,33,35,35,35,35,35,35,35,36,36,36,36,36,38,38,43,43,43,43,43,45,46,54,60,62,63,65,65,80
4,Bihar,8.51,0,0,0,0,0,0,0,0,2,3,3,4,7,9,11,15,15,21,24,29,31,32,32,32,38,39,58,60,64,64,66,66,72,83,85,86,96,113,126,143,170,223,251,277,346,366,403,425,466,481,517,528,535,542,550,579,611,707,749,879,953
5,Chandigarh,20.74,0,0,0,0,0,1,5,5,6,7,7,7,7,8,8,8,13,15,17,18,18,18,18,18,18,18,18,19,19,21,21,21,21,21,21,23,26,26,27,27,27,27,28,36,45,56,68,74,88,94,97,102,119,124,135,146,169,173,181,187,191
6,Dadra and Nagar Haveli,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1
7,Delhi,15.15,7,7,7,8,10,14,20,27,27,30,30,35,36,40,49,72,97,120,152,293,386,445,503,525,576,669,720,903,1069,1154,1510,1561,1578,1640,1707,1893,2003,2081,2156,2248,2376,2514,2625,2918,3108,3314,3439,3515,3738,4122,4549,4898,5104,5532,5980,6318,6542,6923,7233,7639,7998
8,Goa,NaN,0,0,0,0,0,0,0,0,0,0,0,3,3,3,3,5,5,5,5,5,6,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7
9,Gujarat,15.60,0,0,0,0,0,2,7,14,18,30,34,38,43,47,55,63,70,74,87,88,95,108,128,146,175,186,262,378,468,516,572,650,766,929,1099,1376,1743,1939,2178,2407,2624,2815,3071,3301,3548,3774,4082,4395,4721,5054,5428,5804,6245,6625,7013,7403,7797,8195,8542,8904,9268


### State doubling time

In [133]:
dfss = pd.read_csv("../data/"+folder+"/code_data/state_case_death_testpm_ntod.csv")
l = [["state"], ["state_name"], [col for col in dfss.columns if col.endswith("_ntod")]]
columns = [item for sublist in l for item in sublist]
dfss = dfss[columns]
for c in dfss.columns:
    if c!="state" and c!="state_name":
        dfss.rename(columns={c:c.split("_ntod")[0]}, inplace=True)
dfss = dfss.set_index(["state", "state_name"]).dropna(axis=0, how="all").reset_index()
dfss = dfss[dfss["state_name"]!="Chandigarh"]
dfss = dfss[dfss["state_name"]!="Jharkhand"]
dfss["state_name"] = dfss["state_name"] + ", " + dfss["state"]
del dfss["state"]
dfss.to_csv("../data/"+folder+"/code_data/state_doubling_time.csv", index=False)
dfss

,state_name,2020-03-14,2020-03-15,2020-03-16,2020-03-17,2020-03-18,2020-03-19,2020-03-20,2020-03-21,2020-03-22,2020-03-23,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04,2020-05-05,2020-05-06,2020-05-07,2020-05-08,2020-05-09,2020-05-10,2020-05-11,2020-05-12,2020-05-13
0,"Andhra Pradesh, AP",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.64,3.74,3.80,3.16,3.43,4.10,7.52,7.42,9.83,12.21,13.65,13.58,15.36,13.50,12.34,18.23,16.43,15.92,16.47,16.03,10.80,10.73,9.58,9.68,10.65,9.53,9.56,10.14,10.53,10.96,12.47,13.84,15.49,16.38,17.19,17.92,19.04,20.11,22.91,24.85,28.11,27.65,28.98
1,"Assam, AS",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.76,21.38,19.51,NaN,NaN
2,"Bihar, BR",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.50,20.76,21.06,16.33,5.28,4.53,4.85,4.99,4.51,6.18,7.11,8.54,8.64,12.40,11.85,16.99,24.91,34.07,39.16,39.93,35.93,26.35,18.09,9.49,8.51
4,"Delhi, DL",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.18,7.75,3.48,2.75,2.27,2.42,2.93,3.41,6.06,7.34,7.77,8.68,8.16,5.74,6.06,5.65,7.93,20.12,22.90,24.55,25.37,23.65,14.48,14.80,14.66,15.82,15.82,15.40,12.80,11.68,11.33,12.62,15.75,16.70,15.92,15.16,13.30,10.32,9.79,10.15,11.26,13.37,12.45,13.89,14.65,15.15
5,"Gujarat, GJ",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.92,5.83,5.90,7.27,6.57,17.82,18.61,18.37,16.69,16.89,5.53,5.99,5.31,4.88,4.47,5.13,4.20,4.88,5.34,5.41,4.82,4.09,3.59,4.05,4.52,5.08,6.07,7.46,7.75,8.48,9.01,9.65,9.45,9.73,9.75,9.86,9.56,9.86,9.88,10.29,10.69,11.31,11.91,12.80,13.80,14.70,15.60
6,"Haryana, HR",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.96,4.42,4.50,11.91,13.90,21.41,22.50,48.71,54.57,55.22,49.66,51.83,28.24,16.44,46.52,52.65,52.85,55.52,61.22,29.73,25.46,29.74,29.60,36.34,34.71,32.90,27.29,22.12,8.70,9.48,8.52,8.57,11.72,14.10,15.14,17.09,16.46,20.84
7,"Himachal Pradesh, HP",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.98,18.90,14.41,16.63,15.81,23.37
8,"Jammu and Kashmir, JK",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.39,4.16,5.03,6.05,6.19,5.97,9.78,9.65,8.23,8.46,8.66,5.45,5.98,6.82,10.65,11.30,12.58,14.21,16.35,15.31,17.06,18.34,17.19,15.78,15.15,13.22,11.33,12.53,14.42,16.31,17.18,18.22,18.35,17.00,16.00,20.26,19.88,22.57,23.60,28.49,26.41,30.02,26.27,26.10
10,"Karnataka, KA",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.87,4.22,4.61,4.61,9.66,11.39,10.56,11.27,11.84,10.52,9.33,11.25,11.43,12.23,11.97,12.43,12.31,14.16,14.03,12.13,13.19,12.36,9.84,9.08,8.93,15.17,16.28,20.86,26.31,27.60,20.86,20.38,37.83,39.14,42.30,46.22,46.16,26.32,25.38,25.34,21.59,23.22,24.63,25.83,21.45,21.70,19.64,23.37,17.26,18.99
11,"Kerala, KL",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.67,2.87,4.23,4.63,4.63,8.37,8.69,7.89,11.66,12.57,10.25,13.40,16.24,16.91,18.87,22.16,22.93,23.19,24.96,26.66,47.55,59.75,62.33,108.91,111.55,140.38,136.73,157.89,113.53,108.82,59.44,51.79,44.77,44.41,47.20,46.82,57.46,43.49,68.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,139.61,77.16


### Data for top hotspots table

In [134]:
### District cases, deaths, population, cases per million
dfd = pd.read_csv("../data/"+folder+"/derived/district_confirmed_active_recovered_deceased_population_casespm.csv")
dfd = dfd[["district", "state", "state_code", "confirmed", "deceased", "population", "cases_per_million"]]
dfd.rename(columns={"confirmed":"cases", "deceased":"deaths"}, inplace=True)

### District growth rate
dfcg = pd.read_csv("../data/"+folder+"/code_data/districtwise_growth_rate.csv")
for i in range(len(dfcg)):
    dfcg.at[dfcg.index[i], "district"] = dfcg.iloc[i]["district"].split(", ")[0]
l = [["district"], dfcg.columns[-5:]]
columns = [item for sublist in l for item in sublist]
dfcg = dfcg[columns]
for i in range(len(dfcg.columns)):
    col = dfcg.columns[i]
    if col not in ["district", "current_avg_growth_rate"]:
        dfcg.rename(columns={col:"day"+str(i-1)}, inplace=True)
dfcg.insert(1, "current_avg_growth_rate", dfcg[dfcg.columns[-1]])
dfcg

dfht = dfd.set_index("district").join(dfcg.set_index("district")).reset_index()
dfht = dfht[["district", "state", "cases", "deaths", "current_avg_growth_rate", "day0","day1",
            "day2","day3","day4","population", "state_code","cases_per_million"]]
dfht.drop_duplicates(subset=["district"], inplace=True)
dfht.to_csv("../data/"+folder+"/code_data/districtwise_case_death_growth_density.csv", index=False)
dfht

,district,state,cases,deaths,current_avg_growth_rate,day0,day1,day2,day3,day4,population,state_code,cases_per_million
0,Adilabad,Telangana,21.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,708952.0,TG,29.621187
1,Agar Malwa,Madhya Pradesh,13.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MP,NaN
2,Agra,Uttar Pradesh,791.0,24.0,4.79,7.62,7.39,7.01,7.16,4.79,4380793.0,UP,180.560917
3,Ahmadabad,Gujarat,6645.0,446.0,7.39,11.42,9.60,8.58,7.85,7.39,7208200.0,GJ,921.866763
4,Ahmadnagar,Maharashtra,64.0,3.0,11.05,17.02,8.45,10.90,12.71,11.05,4543083.0,MH,14.087350
5,Airport Quarantine,Tamil Nadu,9.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TN,NaN
6,Aizawl,Mizoram,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,404054.0,MZ,2.474917
7,Ajmer,Rajasthan,235.0,5.0,7.41,9.44,9.65,10.35,9.55,7.41,2584913.0,RJ,90.912151
8,Akola,Maharashtra,192.0,12.0,9.70,19.52,20.06,14.28,11.37,9.70,1818617.0,MH,105.574731
9,Alappuzha,Kerala,5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2121943.0,KL,2.356331


### State streamgraph data

In [135]:
df = pd.read_csv("../data/"+folder+"/statewise_daily_confirmed.csv")
del df["TT"]
df["date"] = pd.to_datetime(df["date"], dayfirst=True)
df.to_csv("../data/"+folder+"/code_data/state_streamgraph_data.csv", index=False)
df

,date,AN,AP,AR,AS,BR,CH,CT,DN,DD,DL,GA,GJ,HR,HP,JK,JH,KA,KL,LA,LD,MP,MH,MN,ML,MZ,NL,OR,PY,PB,RJ,SK,TN,TG,TR,UP,UT,WB
0,2020-03-14,0,1,0,0,0,0,0,0,0,7,0,0,14,0,2,0,6,19,0,0,0.0,14,0,0,0,0,0,0,1,3,0,1,1,0,12,0,0
1,2020-03-15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0.0,18,0,0,0,0,0,0,0,1,0,0,2,0,1,0,0
2,2020-03-16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,3,0,0,NaN,6,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0
3,2020-03-17,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,2,0,0,0,0.0,3,0,0,0,0,0,0,0,0,0,0,1,0,2,0,1
4,2020-03-18,0,0,0,0,0,0,0,0,0,2,0,0,1,0,1,0,5,0,8,0,0.0,3,0,0,0,0,1,0,1,3,0,1,8,0,2,1,0
5,2020-03-19,0,2,0,0,0,1,1,0,0,4,0,2,1,0,0,0,1,1,0,0,0.0,4,0,0,0,0,0,0,0,2,0,1,3,0,3,1,0
6,2020-03-20,0,0,0,0,0,4,0,0,0,6,0,5,2,2,0,0,0,12,2,0,4.0,4,0,0,0,0,0,0,1,8,0,0,3,0,4,0,1
7,2020-03-21,0,2,0,0,0,0,0,0,0,7,0,7,2,0,0,0,5,12,3,0,0.0,12,0,0,0,0,0,0,10,7,0,3,2,0,4,0,2
8,2020-03-22,0,1,0,0,2,1,0,0,0,0,0,4,0,0,0,0,6,15,0,0,2.0,10,0,0,0,0,0,0,8,4,0,3,6,0,2,0,3
9,2020-03-23,0,1,0,0,1,1,0,0,0,3,0,12,7,1,0,0,7,28,0,0,0.0,23,0,0,0,0,0,0,2,4,0,3,6,0,2,1,0


### Outbreak free districts

In [136]:
dfcc = pd.read_csv("../data/"+folder+"/code_data/districtwise_case_death_growth_density.csv")
total_population = dfcc["population"].sum()
per_outbreak_free_districts = np.round(len(dfcc[dfcc["cases"]==0])/len(dfcc)*100)
per_outbreak_free_population = np.round(dfcc[dfcc["cases"]==0]["population"].sum()/total_population*100)
df = pd.DataFrame()
for i in range(len(dfcc)):
    district_name = dfcc.iloc[i]["district"].replace(" ","-") + ", " + dfcc.iloc[i]["state_code"]
    df.at[i, "id"] = "district."+district_name
    df.at[i, "value"] = int(dfcc.iloc[i]["cases"])+1
    df.at[i, "district"] = district_name
    df.at[i, "per_outbreak_free_districts"] = per_outbreak_free_districts
    df.at[i, "per_outbreak_free_population"] = per_outbreak_free_population
df["value"] = df["value"].astype(int)
df["per_outbreak_free_districts"] = df["per_outbreak_free_districts"].astype(int)
df["per_outbreak_free_population"] = df["per_outbreak_free_population"].astype(int)
# Shuffling
df = df.sample(frac=1).reset_index(drop=True)
df.to_csv("../data/"+folder+"/code_data/outbreak_free_districts.csv", index=False)
df

,id,value,district,per_outbreak_free_districts,per_outbreak_free_population
0,"district.Tumakuru, KA",12,"Tumakuru, KA",27,10
1,"district.Fatehpur, UP",7,"Fatehpur, UP",27,10
2,"district.Sabar-Kantha, GJ",28,"Sabar-Kantha, GJ",27,10
3,"district.Pithoragarh, UT",1,"Pithoragarh, UT",27,10
4,"district.Rupnagar, PB",57,"Rupnagar, PB",27,10
5,"district.Ramanagara, KA",1,"Ramanagara, KA",27,10
6,"district.Kulgam, JK",73,"Kulgam, JK",27,10
7,"district.Yamunanagar, HR",9,"Yamunanagar, HR",27,10
8,"district.Sambalpur, OR",1,"Sambalpur, OR",27,10
9,"district.Koraput, OR",2,"Koraput, OR",27,10


### State recovery rate

In [137]:
dfc = pd.read_csv("../data/"+folder+"/statewise_daily_confirmed.csv")
del dfc["TT"]
state_code_list = dfc.columns[1:]
dfcc = dfc.set_index("date").cumsum().reset_index()

dfr = pd.read_csv("../data/"+folder+"/statewise_daily_recovered.csv")
del dfr["TT"]
l = [["date"], [col+"_r" for col in dfr.columns if col in state_code_list]]
dfr.columns = [item for sublist in l for item in sublist]
dfr = dfr.set_index("date").cumsum().reset_index()

dfcr = dfcc.set_index("date").join(dfr.set_index("date")).reset_index()
dfrr = dfcc.copy()
for col in state_code_list:
    dfrr[col] = np.round(dfcr[col+"_r"] / dfcr[col],2)
dfrr.dropna(axis=1, how="all", inplace=True)
### Replace nan with -1, handle in color_Scale in javascript
dfrr.fillna(value=-1, inplace=True)
### Concat total case count at top of table
dfrr = pd.concat([pd.DataFrame(dfc.sum(axis=0, numeric_only=True)).transpose(), dfrr], ignore_index=True, axis=0, sort=True)
l = [["date"], [col for col in dfrr.columns[:len(dfrr.columns)-1]]]
columns = [item for sublist in l for item in sublist]
dfrr = dfrr[columns]
dfrr.at[pd.isnull(dfrr["date"]), "date"] = "total_count"
dfrr.to_csv("../data/"+folder+"/code_data/state_recovery_rate.csv", index=False)
dfrr

,date,AN,AP,AR,AS,BR,CH,CT,DD,DL,DN,GA,GJ,HP,HR,JH,JK,KA,KL,LA,LD,MH,ML,MN,MP,MZ,NL,OR,PB,PY,RJ,SK,TG,TN,TR,UP,UT,WB
0,total_count,33.00,2137.00,1.0,80.00,953.00,191.00,59.00,0.0,7998.00,1.0,7.00,9268.00,67.00,793.00,177.00,971.00,959.00,535.00,43.00,0.0,25922.00,13.00,2.0,4173.00,1.0,0.0,538.00,1924.00,13.00,4278.00,0.0,1367.00,9227.00,153.00,3758.00,72.00,2290.00
1,14-Mar-20,-1.00,0.00,-1.0,-1.00,-1.00,-1.00,-1.00,NaN,0.14,-1.0,-1.00,-1.00,-1.00,0.00,-1.00,0.00,0.00,0.16,-1.00,NaN,0.00,-1.00,-1.0,-1.00,-1.0,NaN,-1.00,0.00,-1.00,0.33,NaN,0.00,0.00,-1.00,0.33,-1.00,-1.00
2,15-Mar-20,-1.00,0.00,-1.0,-1.00,-1.00,-1.00,-1.00,NaN,0.29,-1.0,-1.00,-1.00,-1.00,0.00,-1.00,0.00,0.00,0.12,-1.00,NaN,0.00,-1.00,-1.0,-1.00,-1.0,NaN,-1.00,0.00,-1.00,0.75,NaN,0.33,0.00,-1.00,0.31,-1.00,-1.00
3,16-Mar-20,-1.00,0.00,-1.0,-1.00,-1.00,-1.00,-1.00,NaN,0.29,-1.0,-1.00,-1.00,-1.00,0.00,-1.00,0.00,0.00,0.11,-1.00,NaN,0.00,-1.00,-1.0,-1.00,-1.0,NaN,0.00,0.00,0.00,0.75,NaN,0.25,1.00,-1.00,0.31,0.00,-1.00
4,17-Mar-20,-1.00,0.00,-1.0,-1.00,-1.00,-1.00,-1.00,NaN,0.25,-1.0,-1.00,-1.00,-1.00,0.00,-1.00,0.00,0.00,0.11,-1.00,NaN,0.00,-1.00,-1.0,-1.00,-1.0,NaN,0.00,0.00,0.00,0.75,NaN,0.20,1.00,-1.00,0.33,0.00,0.00
5,18-Mar-20,-1.00,0.00,-1.0,-1.00,-1.00,-1.00,-1.00,NaN,0.20,-1.0,-1.00,-1.00,-1.00,0.00,-1.00,0.00,0.00,0.11,0.00,NaN,0.00,-1.00,-1.0,-1.00,-1.0,NaN,0.00,0.00,0.00,0.43,NaN,0.08,0.50,-1.00,0.29,0.00,0.00
6,19-Mar-20,-1.00,0.00,-1.0,-1.00,-1.00,0.00,0.00,NaN,0.21,-1.0,-1.00,0.00,-1.00,0.00,-1.00,0.00,0.00,0.11,0.00,NaN,0.00,-1.00,-1.0,-1.00,-1.0,NaN,0.00,0.00,0.00,0.33,NaN,0.06,0.33,-1.00,0.45,0.00,0.00
7,20-Mar-20,-1.00,0.00,-1.0,-1.00,-1.00,0.00,0.00,NaN,0.25,-1.0,-1.00,0.00,0.00,0.00,-1.00,0.00,0.07,0.08,0.00,NaN,0.00,-1.00,-1.0,0.00,-1.0,NaN,0.00,0.00,0.00,0.18,NaN,0.05,0.33,-1.00,0.38,0.00,0.00
8,21-Mar-20,-1.00,0.00,-1.0,-1.00,-1.00,0.00,0.00,NaN,0.19,-1.0,-1.00,0.00,0.00,0.00,-1.00,0.00,0.05,0.06,0.00,NaN,0.00,-1.00,-1.0,0.00,-1.0,NaN,0.00,0.00,0.00,0.12,NaN,0.05,0.17,-1.00,0.32,0.00,0.00
9,22-Mar-20,-1.00,0.00,-1.0,-1.00,0.00,0.00,0.00,NaN,0.19,-1.0,-1.00,0.00,0.00,0.00,-1.00,0.00,0.04,0.04,0.00,NaN,0.00,-1.00,-1.0,0.00,-1.0,NaN,0.00,0.00,0.00,0.11,NaN,0.04,0.11,-1.00,0.30,0.00,0.00


### Cluster table and cluster network map

In [138]:
from shutil import copyfile
src = "../data/extracted_data_api/code_data/cluster_table.csv"
dest = "../data/"+folder+"/code_data/cluster_table.csv"
copyfile(src, dest)

src = "../data/extracted_data_api/code_data/cluster_network_map.csv"
dest = "../data/"+folder+"/code_data/cluster_network_map.csv"
copyfile(src, dest)

'../data/09052020/code_data/cluster_network_map.csv'

### 4*9 grid state wise plots

In [ ]:
### For missing dates
# District time series available till 13th April, current district confirmed cases available, interpolate in between
# For future dates, two options
###### 1. Diff from daily howIndiaLives data 
###### 2. District daily data from covid19india api

In [180]:
init_date = pd.datetime(2020,3,15)

df = pd.read_csv("../data/09052020/derived/districtwise_time_series.csv")
df["date"] = pd.to_datetime(df["date"])
df = df[df["date"]>=init_date]
dfsc = pd.read_csv("../data/extracted_data_api/state_codes.csv")
df = df.set_index("state").join(dfsc.set_index("state")).reset_index()

### MAp district names
for i in range(len(df)):
    district_name = df.iloc[i]["district"]
    state_name = df.iloc[i]["state"]
    
    if district_name in district_name_mapping:
        district_name_mapped = district_name_mapping[district_name]
        district_name = district_name_mapped

    if district_name in district_name_resolution:
        district_name_resolved = resolve_district_name(state_name, district_name)
        district_name = district_name_resolved
    
    df.at[i, "district"] = district_name
    

    
district_list = df["district"].unique()
dfd = pd.DataFrame()
for i in range(len(district_list)):
    district_name = district_list[i]

    if district_name==district_name:
        dft = df[df["district"]==district_name].copy()
        dft["count"] = dft["count"].cumsum()
        state_name = dft["state"].unique()[0]
        state_code = dft["state_code"].unique()[0]
        dft.reset_index(inplace=True)
        del dft["index"], dft["state"], dft["district"], dft["state_code"]
        dft.rename(columns={"count":district_name}, inplace=True)
        dft[district_name] = dft[district_name].astype(str)
        dft["date"] = pd.to_datetime(dft["date"]).dt.date
        dft.at[-1, "date"] = "state"
        dft.at[-1, district_name] = state_name
        dft.at[-2, "date"] = "state_code"
        dft.at[-2, district_name] = state_code
        dft.sort_index(inplace=True)
        dft = dft.transpose()
        new_header = dft.iloc[0]
        dft = dft[1:]
        dft.columns = new_header
        dft.reset_index(inplace=True)
        dft.rename(columns={"index":"district"}, inplace=True)

        if i==0:
            dfd = dft.copy()
        else:
            dfd = dfd.append(dft)

dfd.to_csv("../data/"+folder+"/code_data/districtwise_daily_case_count.csv", index=False)

### Outbreak spread map

In [ ]:
### For missing dates
# District time series available till 13th April, current district confirmed cases available, interpolate in between
# For future dates, two options
###### 1. Diff from daily howIndiaLives data 
###### 2. District daily data from covid19india api

### Country case count

In [139]:
country_case_count_thresh = 10000
start_counting_from_case = 200
df = pd.read_csv("../data/"+folder+"/owid/owid-covid-data.csv")[["location", "date", "total_cases"]]
df = df[df["location"] != "World"]
df = df[df["location"] != "International"]
df = df.groupby("location").filter(lambda x: x.tail(1)["total_cases"]>country_case_count_thresh)
df = df[df["total_cases"] >= start_counting_from_case]

dfca = pd.DataFrame()
country_list = df["location"].unique()
for i in range(len(country_list)):
    country_name = country_list[i]
    
    dfc = df[df["location"]==country_name].copy()
    dfc.reset_index(inplace=True)
    del dfc["index"]
    dfc.reset_index(inplace=True)
    dfc.rename(columns={"index":"day", "total_cases":country_name}, inplace=True)
    del dfc["date"], dfc["location"]
    dfc["day"] = np.round(dfc["day"]) #.astype(int)
    dfc
    
    if i==0:
        dfca = dfc.copy()
    else:
        dfca = dfca.set_index("day").join(dfc.set_index("day"), how="outer").reset_index()
        dfca["day"] = np.round(dfca["day"]) #.astype(int)
dfca = dfca.transpose()
new_header = dfca.iloc[0]
dfca = dfca[1:]
dfca.columns = new_header
dfca.reset_index(inplace=True)
dfca.rename(columns={"index":"country"}, inplace=True)
idx = 0
for col in dfca.columns:
    if col!="country":
        dfca.rename(columns={col:int(idx)}, inplace=True)
        idx+=1
dfca.to_csv("../data/"+folder+"/code_data/country_case_count.csv", index=False)
dfca

day,country,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,...,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115
0,United Arab Emirates,248.0,333.0,333.0,405.0,468.0,468.0,611.0,664.0,664.0,1024.0,1024.0,1505.0,1799.0,2076.0,2359.0,2659.0,2990.0,3360.0,3736.0,4123.0,4521.0,4933.0,5365.0,5825.0,5825.0,6302.0,6781.0,7265.0,7755.0,8238.0,8756.0,9281.0,9813.0,10349.0,10839.0,11380.0,11929.0,12481.0,13038.0,13599.0,14163.0,14730.0,15192.0,15738.0,16240.0,16793.0,17417.0,18198.0,18878.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Austria,246.0,361.0,504.0,655.0,860.0,1016.0,1332.0,1646.0,2196.0,2649.0,3024.0,3631.0,4486.0,5282.0,5888.0,7029.0,7697.0,8291.0,8813.0,9618.0,10182.0,10711.0,11129.0,11525.0,11766.0,11983.0,12297.0,12640.0,12969.0,13248.0,13560.0,13807.0,13937.0,14043.0,14234.0,14370.0,14448.0,14603.0,14662.0,14710.0,14783.0,14833.0,14924.0,14985.0,15068.0,15134.0,15189.0,15256.0,15314.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Belgium,200.0,239.0,267.0,314.0,399.0,559.0,689.0,886.0,1085.0,1243.0,1486.0,1795.0,2257.0,2815.0,3401.0,3743.0,4269.0,4937.0,6235.0,7284.0,9134.0,10836.0,11899.0,12775.0,13964.0,15348.0,16770.0,18431.0,19691.0,20814.0,22194.0,23403.0,24983.0,26667.0,28018.0,29647.0,30589.0,31119.0,33573.0,34809.0,36138.0,37183.0,38496.0,39983.0,40956.0,41889.0,42797.0,44293.0,45325.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Bangladesh,218.0,330.0,424.0,482.0,621.0,803.0,1012.0,1231.0,1572.0,1838.0,2144.0,2456.0,2948.0,3382.0,3772.0,4186.0,4689.0,4998.0,5416.0,5913.0,6462.0,7103.0,7667.0,8238.0,8790.0,9455.0,10143.0,10929.0,11719.0,12425.0,13134.0,13770.0,14657.0,15691.0,16660.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Belarus,254.0,351.0,394.0,564.0,700.0,861.0,1066.0,1486.0,1981.0,2226.0,2578.0,2919.0,3281.0,3728.0,4204.0,4779.0,4779.0,4779.0,6264.0,6723.0,7281.0,8022.0,8773.0,9590.0,10463.0,11289.0,12208.0,13181.0,14027.0,14917.0,15828.0,16705.0,17489.0,18350.0,19255.0,20168.0,21101.0,22052.0,22973.0,23906.0,24873.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Brazil,200.0,234.0,291.0,428.0,621.0,904.0,1128.0,1546.0,1891.0,2201.0,2433.0,2915.0,3417.0,3904.0,4256.0,4579.0,5717.0,6836.0,7910.0,9056.0,10278.0,11130.0,12056.0,13717.0,15927.0,17857.0,19638.0,20727.0,22169.0,23430.0,25262.0,28320.0,30425.0,33682.0,36599.0,38654.0,40581.0,43079.0,45757.0,49492.0,52995.0,58509.0,61888.0,66501.0,71886.0,78162.0,85380.0,91589.0,96559.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Canada,244.0,304.0,424.0,569.0,690.0,846.0,971.0,1302.0,1430.0,1646.0,1959.0,3385.0,4018.0,4675.0,5386.0,6255.0,7424.0,8536.0,9595.0,11268.0,12519.0,13882.0,15496.0,16653.0,17883.0,19274.0,20748.0,22133.0,23301.0,24365.0,25663.0,27046.0,28364.0,30081.0,31872.0,33341.0,34777.0,36823.0,38413.0,40179.0

### Country testing rate

In [140]:
country_case_count_thresh = 10000
start_counting_from_case = 200
df = pd.read_csv("../data/"+folder+"/owid/owid-covid-data.csv")[["location", "date", "total_cases", "total_cases_per_million", "total_tests_per_thousand"]]
df = df[df["location"] != "World"]
df = df[df["location"] != "International"]
df = df.groupby("location").filter(lambda x: x.tail(1)["total_cases"]>country_case_count_thresh)
df = df.dropna(subset=["total_cases_per_million", "total_tests_per_thousand"])
df = df.groupby('location').nth(-1).reset_index()
del df["date"]
df.rename(columns={"location":"country"}, inplace=True)
df.to_csv("../data/"+folder+"/code_data/country_testing_rate.csv", index=False)
df

,country,total_cases,total_cases_per_million,total_tests_per_thousand
0,Austria,15874,1762.524,36.564
1,Bangladesh,15691,95.276,0.830
2,Belarus,23906,2529.917,30.102
3,Belgium,52596,4538.200,42.095
4,Brazil,38654,181.850,0.623
5,Canada,69981,1854.187,30.356
6,Chile,30063,1572.644,15.868
7,Colombia,11063,217.421,3.124
8,Denmark,10513,1815.026,57.709
9,Ecuador,29538,1674.199,2.622


### Scrollytelling data

In [207]:
df = pd.read_csv("../data/09052020/one_time/full_data_patient_level_withDistrictMapped_withStatusMapped.csv")
df = df[["date", "district", "state", "state_code", "status", "status_change_date"]]
df["date"] = pd.to_datetime(df["date"], dayfirst=True).dt.date
df["status_change_date"] = pd.to_datetime(df["status_change_date"], dayfirst=True).dt.date
df = df[::10]
df.reset_index(inplace=True)
del df["index"]
df.reset_index(inplace=True)
df.rename(columns={"index":"id"}, inplace=True)
#df["district_lat"] = np.nan
#df["district_long"] = np.nan
#df["state_lat"] = np.nan
#df["state_long"] = np.nan
df.at[df["state"]=="Delhi", "district"] = "Delhi"
df

dfdll = pd.read_csv("../data/extracted_data_api/code_data/district_center_bounds.csv")
dfdll.at[dfdll["district"]=="New Delhi", "district"] = "Delhi"
dfsll = pd.read_csv("../data/extracted_data_api/code_data/state_center_bounds.csv")
dfsll.rename(columns={"location":"state"}, inplace=True)

####################### Map district names ###############################
district_list = df["district"].unique()
for i in range(len(district_list)):
    district_name = district_list[i]  # df.iloc[i]["district"]
    district_name_found = district_list[i]  # df.iloc[i]["district"]
    if district_name in data_to_map_district_name_mapping:
        district_name = data_to_map_district_name_mapping[district_name]

    if district_name in district_name_mapping:
        district_name = district_name_mapping[district_name]

    if district_name in district_name_wiki_mapping:
        district_name = district_name_wiki_mapping[district_name]

    if district_name in district_name_resolution:
        district_name = district_name+" ("+dfd["state"].unique()[0]+")"

    if district_name=="Aurangabad(MH)":
        district_name = "Aurangabad (Maharashtra)"

    df.at[df["district"]==district_name_found, "district"] = district_name
##########################################################################

df = df.set_index("district").join(dfdll[["district", "latitude", "longitude"]].set_index("district")).reset_index()
df.rename(columns={"latitude":"district_lat", "longitude":"district_long"}, inplace=True)
df = df[~pd.isnull(df["district_lat"])]
df = df.set_index("state").join(dfsll[["state", "latitude", "longitude"]].set_index("state")).reset_index()
df.rename(columns={"latitude":"state_lat", "longitude":"state_long"}, inplace=True)

df.sort_values("date", inplace=True)
df

,state,district,id,date,state_code,status,status_change_date,district_lat,district_long,state_lat,state_long
1915,Kerala,Thrissur,0,2020-01-30,KL,Recovered,2020-04-09,10.471392,76.315042,10.352874,76.512040
1658,Haryana,Gurgaon,1,2020-03-04,HR,Recovered,2020-04-01,28.368813,76.941136,29.000000,76.000000
4384,Rajasthan,Jaipur,2,2020-03-04,RJ,Recovered,2020-03-15,26.976868,75.718689,26.810578,73.768455
265,Delhi,Delhi,3,2020-03-06,DL,Recovered,2020-03-15,28.578610,77.147670,28.651718,77.221939
1745,Jammu and Kashmir,Jammu,4,2020-03-09,JK,Recovered,2020-04-29,32.798321,74.749358,34.047656,76.797289
1805,Karnataka,Bangalore,5,2020-03-10,KA,Recovered,2020-04-21,12.952255,77.594187,14.520390,75.722352
3678,Maharashtra,Pune,6,2020-03-10,MH,Hospitalized,2020-03-10,18.570765,74.068534,19.531932,76.055457
3579,Maharashtra,Nagpur,7,2020-03-11,MH,Recovered,2020-03-28,21.178173,79.085043,19.531932,76.055457
5492,Uttar Pradesh,Ghaziabad,8,2020-03-12,UP,Recovered,2020-03-14,28.767770,77.475813,27.130334,80.859666
5350,Uttar Pradesh,Agra,9,2020-03-13,UP,Recovered,2020-03-19,27.036633,78.075949,27.130334,80.859666


In [208]:
### Assign day_id
df["date"] = pd.to_datetime(df["date"], dayfirst=True).dt.date
df.sort_values("date", inplace=True)
start_date = df.head(1)["date"].values[0]
end_date = df.tail(1)["date"].values[0]
df["day_id"] = -1
cdt = start_date
while (cdt <= end_date):
    current_date_indx = df[df["date"]==cdt].index
    for i in range(len(current_date_indx)):
        df.at[current_date_indx[i], "day_id"] = i
    cdt+=timedelta(1)

### Assign district_id
df["district_id"] = -1
district_list = df["district"].unique()
for i in range(len(district_list)):
    district_name = district_list[i]
    dfd = df[df["district"]==district_name].copy()
    current_indx = dfd.index
    num_cases = len(dfd)
    for j in range(num_cases):
        df.at[current_indx[j], "district_id"] = int(j)

### Assign state
df["state_id"] = -1
state_list = df["state"].unique()
for i in range(len(state_list)):
    state_name = state_list[i]
    dfd = df[df["state"]==state_name].copy()
    current_indx = dfd.index
    num_cases = len(dfd)
    for j in range(num_cases):
        df.at[current_indx[j], "state_id"] = int(j)
df


dfdc = df.groupby("district").count()["id"].reset_index()
dfdc.rename(columns={"id":"num_cases_in_district"}, inplace=True)
df = df.set_index("district").join(dfdc.set_index("district")).reset_index()

dfsc = df.groupby("state").count()["id"].reset_index()
dfsc.rename(columns={"id":"num_cases_in_state"}, inplace=True)
df = df.set_index("state").join(dfsc.set_index("state")).reset_index()

df.sort_values("date", inplace=True)
df.to_csv("../data/09052020/code_data/scroll_data.csv", index=False)
df

,state,district,id,date,state_code,status,status_change_date,district_lat,district_long,state_lat,state_long,day_id,district_id,state_id,num_cases_in_district,num_cases_in_state
1915,Kerala,Thrissur,0,2020-01-30,KL,Recovered,2020-04-09,10.471392,76.315042,10.352874,76.512040,0,0,0,4,50.0
1658,Haryana,Gurgaon,1,2020-03-04,HR,Recovered,2020-04-01,28.368813,76.941136,29.000000,76.000000,0,0,0,10,62.0
4384,Rajasthan,Jaipur,2,2020-03-04,RJ,Recovered,2020-03-15,26.976868,75.718689,26.810578,73.768455,1,0,0,114,331.0
265,Delhi,Delhi,3,2020-03-06,DL,Recovered,2020-03-15,28.578610,77.147670,28.651718,77.221939,0,0,0,633,633.0
1745,Jammu and Kashmir,Jammu,4,2020-03-09,JK,Recovered,2020-04-29,32.798321,74.749358,34.047656,76.797289,0,0,0,6,77.0
1805,Karnataka,Bangalore,5,2020-03-10,KA,Recovered,2020-04-21,12.952255,77.594187,14.520390,75.722352,0,0,0,18,68.0
3678,Maharashtra,Pune,6,2020-03-10,MH,Hospitalized,2020-03-10,18.570765,74.068534,19.531932,76.055457,1,0,0,221,1855.0
3579,Maharashtra,Nagpur,7,2020-03-11,MH,Recovered,2020-03-28,21.178173,79.085043,19.531932,76.055457,0,0,1,23,1855.0
5492,Uttar Pradesh,Ghaziabad,8,2020-03-12,UP,Recovered,2020-03-14,28.767770,77.475813,27.130334,80.859666,0,0,0,12,309.0
5350,Uttar Pradesh,Agra,9,2020-03-13,UP,Recovered,2020-03-19,27.036633,78.075949,27.130334,80.859666,0,0,1,73,309.0


In [62]:
dfll = pd.read_csv("../data/extracted_data_api/code_data/district_center_bounds.csv")

district_list = df["district"].unique()
dfcp = pd.DataFrame()
idx = 0
for i in range(len(district_list)):
    district_name = district_list[i]
    if district_name==district_name:
        dfd = df[df["district"]==district_name].copy()
        start_date = dfd.head(1)["date"].values[0]
        end_date = dfd.tail(1)["date"].values[0]
        district_total_case_count = int(len(dfd))
        max_district_radius = np.log10(district_total_case_count) #district_total_case_count/1000 #np.log2(district_total_case_count)/10
        cdt = pd.to_datetime(start_date).date()

        if district_name in data_to_map_district_name_mapping:
            district_name = data_to_map_district_name_mapping[district_name]

        if district_name in district_name_mapping:
            district_name = district_name_mapping[district_name]
        
        if district_name in district_name_wiki_mapping:
            district_name = district_name_wiki_mapping[district_name]
        
        if district_name in district_name_resolution:
            district_name = district_name+" ("+dfd["state"].unique()[0]+")"
            
        if district_name in district_list_not_in_map:
            break

        if district_name=="Aurangabad(MH)":
            district_name = "Aurangabad (Maharashtra)"

        if district_name=="Delhi":
            dflld = dfll[dfll["district"]=="New Delhi"].copy()
        else:
            dflld = dfll[dfll["district"]==district_name].copy()

        if len(dflld)==0:
            print(district_name + ": No lat long data")
        elif len(dflld)>1:
            print(district_name + ": More than one lat long data")
        else:
            district_center_lat = dflld["latitude"].values[0]
            district_center_long = dflld["longitude"].values[0]

            while (cdt <= end_date):
                num_cases = len(dfd[dfd["date"]==cdt])
                pid_array = dfd[dfd["date"]==cdt]["id"].values
                
                for c in range(num_cases):
                    theta = random.uniform(0, 360)
                    radius = c/num_cases*max_district_radius #random.uniform(0, max_district_radius)
                    patient_district_lat = district_center_lat #+ radius*np.cos(theta*np.pi/180) 
                    patient_district_long = district_center_long #+ radius*np.sin(theta*np.pi/180) 

                    pid = pid_array[c]
                    df.at[df["id"]==pid, "district_lat"] = patient_district_lat
                    df.at[df["id"]==pid, "district_long"] = patient_district_long
                    
                    idx+=1

                cdt += timedelta(days=1)


Ranipet: No lat long data
Chengalpattu: No lat long data


In [52]:
dfll = pd.read_csv("../data/extracted_data_api/code_data/state_center_bounds.csv")

state_list = df["state"].unique()
dfcp = pd.DataFrame()
idx = 0
for i in range(len(state_list)):
    state_name = state_list[i]
    if state_name==state_name:
        dfd = df[df["state"]==state_name].copy()
        start_date = dfd.head(1)["date"].values[0]
        end_date = dfd.tail(1)["date"].values[0]
        state_total_case_count = int(len(dfd))
        max_state_radius = np.log10(state_total_case_count) #state_total_case_count/1000 #np.log2(district_total_case_count)/10
        cdt = pd.to_datetime(start_date).date()

        dflld = dfll[dfll["location"]==state_name].copy()

        if len(dflld)==0:
            print(state_name + ": No lat long data")
        elif len(dflld)>1:
            print(state_name + ": More than one lat long data")
        else:
            state_center_lat = dflld["latitude"].values[0]
            state_center_long = dflld["longitude"].values[0]

            while (cdt <= end_date):
                num_cases = len(dfd[dfd["date"]==cdt])
                pid_array = dfd[dfd["date"]==cdt]["id"].values
                
                for c in range(num_cases):
                    theta = random.uniform(0, 360)
                    radius = c/num_cases*max_state_radius #random.uniform(0, max_district_radius)
                    patient_state_lat = state_center_lat + radius*np.cos(theta*np.pi/180) 
                    patient_state_long = state_center_long + radius*np.sin(theta*np.pi/180) 

                    pid = pid_array[c]
                    df.at[df["id"]==pid, "state_lat"] = patient_state_lat
                    df.at[df["id"]==pid, "state_long"] = patient_state_long
                    
                    idx+=1

                cdt += timedelta(days=1)

df.to_csv("../data/09052020/code_data/outbreak_spread_case_location.csv", index=False)

Dadra and Nagar Haveli and Daman and Diu: No lat long data


### State testing vs ncases - scroll

In [214]:
dfdp = pd.read_csv("../data/wiki_scrape/district_population_wiki_withNameMapped.csv")
dfdp = dfdp.groupby("state").sum()["population"].reset_index()

dfsll = pd.read_csv("../data/extracted_data_api/code_data/state_center_bounds.csv")
dfsll.rename(columns={"location":"state"}, inplace=True)

df = pd.read_csv("../data/"+folder+"/code_data/state_case_death_testpm_ntod.csv")
data_till_date = "2020-05-13"
df = df[["state", "state_name", data_till_date+"_cases", data_till_date+"_deaths", data_till_date+"_testspm", data_till_date+"_ntod"]]
df.columns = ["state", "state_name", "cases", "deaths", "testspm", "ntod"]
#df = df[df["cases"]>=100]
df.rename(columns={"state":"state_code", "state_name":"state"}, inplace=True)
df = df.set_index("state").join(dfdp.set_index("state")).reset_index()
df["casespm"] = df["cases"] / df["population"] * 1e6
df["casespm"] = np.round(df["casespm"], 2)

df = df.set_index("state").join(dfsll[["state", "latitude", "longitude"]].set_index("state")).reset_index()
df.rename(columns={"latitude":"state_lat", "longitude":"state_long"}, inplace=True)
df.sort_values("cases", inplace=True, ascending=False)
df.reset_index(inplace=True)
del df["index"]
df.reset_index(inplace=True)
df.rename(columns={"index":"state_id"}, inplace=True)
df.to_csv("../data/"+folder+"/code_data/scroll_state_data.csv", index=False)
df

,state_id,state,state_code,cases,deaths,testspm,ntod,population,casespm,state_lat,state_long
0,0,Maharashtra,MH,25922.0,974.0,2002.80,11.98,115369105.0,224.69,19.531932,76.055457
1,1,Gujarat,GJ,9268.0,566.0,1843.40,15.60,64845397.0,142.92,22.415408,72.031497
2,2,Tamil Nadu,TN,9227.0,64.0,3530.04,8.45,75547848.0,122.13,10.909433,78.366535
3,3,Delhi,DL,7998.0,106.0,6751.57,15.15,16787941.0,476.41,28.651718,77.221939
4,4,Rajasthan,RJ,4278.0,121.0,2837.08,19.85,68621012.0,62.34,26.810578,73.768455
5,5,Madhya Pradesh,MP,4173.0,233.0,1183.27,15.88,72597832.0,57.48,23.969928,79.394870
6,6,Uttar Pradesh,UP,3758.0,86.0,707.43,23.50,205867440.0,18.25,27.130334,80.859666
7,7,West Bengal,WB,2290.0,207.0,620.34,11.46,92903903.0,24.65,22.996495,87.685588
8,8,Andhra Pradesh,AP,2137.0,47.0,4074.54,28.98,49378776.0,43.28,15.924091,80.186381
9,9,Punjab,PB,1924.0,32.0,1508.51,50.34,30510873.0,63.06,30.929321,75.500484


In [105]:
state_name_mapping = {'odisha':'Odisha', 'telangana':'Telangana', 'meghalaya':'Meghalaya','karnataka':'Karnataka',
                     'haryana':'Haryana','bihar':'Bihar','andhrapradesh':'Andhra Pradesh','jammukashmir':'Jammu and Kashmir',
                     'westbengal':'West Bengal','kerala':'Kerala','chhattisgarh':'Chhattisgarh','andamannicobarislands':'Andaman and Nicobar Islands',
                     'jharkhand':'Jharkhand','ladakh':'Ladakh','uttarpradesh':'Uttar Pradesh','mizoram':'Mizoram','lakshadweep':'Lakshadweep',
                     'nagaland':'Nagaland','tamilnadu':'Tamil Nadu','dadranagarhaveli':'Dadra and Nagar Haveli','delhi':'Delhi',
                     'puducherry':'Puducherry','madhyapradesh':'Madhya Pradesh','arunachalpradesh':'Arunachal Pradesh','uttarakhand':'Uttarakhand',
                     'manipur':'Manipur','tripura':'Tripura','gujarat':'Gujarat','goa':'Goa','assam':'Assam','maharashtra':'Maharashtra',
                     'punjab':'Punjab','sikkim':'Sikkim','rajasthan':'Rajasthan','chandigarh':'Chandigarh','himachalpradesh':'Himachal Pradesh'}

state_name_mapping_howindialives = {"Jammu & Kashmir":"Jammu and Kashmir",
                                    "A& N Islands":"Andaman and Nicobar Islands",
                                    "D & N Haveli":"Dadra and Nagar Haveli"}

district_name_mapping = {"Jagitial":"Jagtial", "Jangoan":"Jangaon", "Kumuram Bheem Asifabad":"Komaram Bheem Asifabad",
                         "Mahabubnagar": "Mahbubnagar", "Ranga Reddy": "Rangareddy", "Yadadri Bhuvanagiri":"Yadadri Bhongir",
                         "Jayashankar": "Jayashankar Bhupalapally",
                         "West Khasi Hills":"West khasi Hills", "East Jaintia Hills":"East Jainta Hills",
                        "Kalaburagi":"Gulbarga", "Belagavi":"Belgaum", "Bagalkote":"Bagalkot", "Ballari":"Bellary",
                        "Shivamogga":"Shimoga", "Chikkamagaluru":"Chikmagalur", "Bengaluru Rural":"Bangalore Rural",
                        "Bengaluru":"Bangalore", "Bengaluru Urban":"Bangalore", "Chamarajanagara":"Chamarajanagar", "Tumakuru":"Tumkur", "Mysuru":"Mysore","Vijayapura":"Bijapur",
                        "Gurugram":"Gurgaon", "Mahendragarh":"Narnaul","Charki Dadri":"Ch-Dadri",
                        "West Champaran": "Pashchim Champaran", "East Champaran":"Purba Champaran", "Kaimur Bhabhua":"Kaimur (Bhabua)", "Kaimur Bhabua":"Kaimur (Bhabua)",
                        "S.P.S. Nellore":"Nellore", "Chittoor":"chittoor",
                        "South 24 Parganas":"South Twenty Four Parganas", "Medinipur West":"Paschim Medinipur", "Medinipur East":"Purba Medinipur",
                        "Darjeeling":"Darjiling", "Purulia":"Puruliya", "North 24 Parganas":"North Twenty Four Parganas", "Hooghly":"Hugli",
                        "Howrah":"Haora", "Cooch Behar":"Koch Bihar",
                        "Gariaband":"Gariyaband", "Kabeerdham":"Kawardha", "Janjgir Champa":"Janjgir - Champa", "Bametara":"Bemetara",
                        "Baloda Bazar":"Balodabazar",
                        "North and Middle Andaman":"North & Middle Andman", "South Andaman":"South Andman","Nicobars":"Nicobar",
                        "Saraikela-kharsawan":"Saraikela-Kharsawan",
                        "Leh":"Leh(Ladakh)",
                        "Amroha":"J P Nagar (Amroha)", "Kheri":"Lakhimpur Kheri", "Kasganj":"Kanshiram Nagar", "Prayagraj":"Allahabad",
                        "Bhadohi":"Sant Ravidas Nagar (Bhadohi)", "Amethi":"C S M Nagar (Amethi)",
                        "Thoothukudi":"Thoothukkudi","Dadra and Nagar Haveli":"Uni District UT", 
                        "Khargone":"Khargone (West Nimar)", "Khandwa":"Khandwa (East Nimar)", "Ashoknagar":"Ashok Nagar",
                        "Kra Daadi":"Kra daadi", "Lower Dibang Valley":"Lower Dibang valley", "Upper Dibang Valley":"Dibang Valley",
                        "Haridwar":"Hardwar", "Noney":"None", "Pherzawl":"Pherjawl", "Sipahijala":"Sepahijala District","Gomati":"Gomati District",
                        "Khowai":"Khowai District", "Unokoti":"Unakoti District", "The Dangs":"Dang", "Chota Udaipur":"Chhota Udepur",
                        "Aravalli":"Arvalli",
                        "S.A.S. Nagar":"Mohali", "Sri Muktsar Sahib":"Muktsar","Jalore":"Jalor",
                        "Jhunjhunu":"Jhunjhunun", "Lahul and Spiti":"Lahul & Spiti","North  District":"North District"
                        }


district_name_resolution = ["Aurangabad", "Bijapur", "Raigarh", "Bilaspur", "Balrampur", "Hamirpur", "Pratapgarh"]

"""
district_name_unmapped_old = ["Muzaffarabad", "Mirpur", "Shi Yomi", "Pakke Kessang", "Pakke-Kessang", "Lepa Rada", "Pauri Garhwal",
                         "Charaideo", "Majuli", "Biswanath", "Hojai", "South Salmara Mancachar",
                         "Shahid Bhagat Singh Nagar", "West Karbi Anglong"]
"""

district_name_wiki_mapping = {"Sri Potti Sriramulu Nellore":"Nellore", "Kadapa":"Y.S.R.",
    "Dantewada":"Dakshin Bastar Dantewada", "Janjgir-Champa":"Janjgir - Champa","Kabirdham":"Kawardha", "Kanker":"Uttar Bastar Kanker",
    "Ahmedabad":"Ahmadabad", "Banaskantha": "Banas Kantha", "Dahod":"Dohad", "Devbhoomi Dwarka":"Devbhumi Dwarka",
    "Kutch":"Kachchh", "Mehsana":"Mahesana", "Panchmahal":"Panch Mahals", "Sabarkantha":"Sabar Kantha",
    "Charkhi Dadri":"Ch-Dadri", "Hissar":"Hisar", "Yamuna Nagar":"Yamunanagar",
    "Lahaul and Spiti":"Lahul & Spiti",
    "East Singhbhum":"Purbi Singhbhum", "West Singhbhum":"Pashchimi Singhbhum", "Hazaribag":"Hazaribagh", "Koderma":"Kodarma", "Seraikela Kharsawan":"Saraikela-Kharsawan",
    "Bangalore Urban":"Bangalore","Chamarajnagar":"Chamarajanagar","Chikkaballapur":"Chikkaballapura",
    "Narsinghpur":"Narsimhapur", "Ahmednagar":"Ahmadnagar", "Beed":"Bid", "Buldhana":"Buldana", "Gondia":"Gondiya", "Mumbai City":"Mumbai","Mumbai suburban":"Mumbai Suburban",
    "Raigad":"Raigarh (Maharashtra)","Ri Bhoi":"Ribhoi", "Angul":"Anugul", "Boudh (Bauda)":"Baudh", "Boudh":"Baudh", "Bargarh (Baragarh)":"Bargarh", "Balasore":"Baleshwar",
    "Debagarh (Deogarh)":"Debagarh", "Jajpur":"Jajapur", "Jagatsinghpur":"Jagatsinghapur","Kendujhar (Keonjhar)":"Kendujhar", "Nabarangpur":"Nabarangapur", "Subarnapur (Sonepur)":"Subarnapur",
    "Chittorgarh":"Chittaurgarh", "Dholpur":"Dhaulpur", "East Sikkim":"East District", "North Sikkim":"North District", "South Sikkim":"South District", "West Sikkim":"West District",
    "Kanchipuram":"Kancheepuram", "Kanyakumari":"Kanniyakumari", "Nilgiris":"The Nilgiris", "Tirupur":"Tiruppur",
    "Tiruvallur":"Thiruvallur","Tiruvarur":"Thiruvarur", "Komaram Bheem":"Komaram Bheem Asifabad","Jayashankar Bhupalpally":"Jayashankar Bhupalapally", "Medchal-Malkajgiri":"Medchal Malkajgiri",
    "Sepahijala":"Sepahijala District", "Bagpat":"Baghpat", "Barabanki":"Bara Banki", "Gautam Buddh Nagar":"Gautam Buddha Nagar",
    "Maharajganj":"Mahrajganj", "Raebareli":"Rae Bareli", "Shravasti":"Shrawasti","Bandipora":"Bandipore", "Baramulla":"Baramula",
    "Poonch":"Punch", "Shopian":"Shupiyan","Mahé":"Mahe","Pondicherry":"Puducherry",
    "Kaimur":"Kaimur (Bhabua)", "Sahibzada Ajit Singh Nagar":"Mohali","Jalore":"Jalor"
}

district_name_unmapped = ["Muzaffarabad", "Mirpur", "Shi Yomi", "Pakke Kessang", "Pakke-Kessang", "Lepa Rada", "Pauri Garhwal",
                         "Charaideo", "Majuli", "Biswanath", "Hojai", "South Salmara Mancachar", "South Salmara", "Bishwanath",
                         "Shahid Bhagat Singh Nagar", "West Karbi Anglong",
                         "Gaurela-Pendra-Marwahi", "Chachaura-Binaganj", "Maihar", "Niwari", "Nagda", "Noklak",
                         "Sahibzada Ajit Singh Nagar", "Jalore", "Chengalpattu", "Kallakurichi", "Mayiladuthurai",
                         "Ranipet","Tenkasi","Tirupattur", "Daman", "Diu", "Niwari", "Tirupathur"]

def resolve_district_name(state_name, district_name):
    if ((state_name=="bihar") and (district_name=="Aurangabad")):
        district_name = "Aurangabad (Bihar)"
    if ((state_name=="Bihar") and (district_name=="Aurangabad")):
        district_name = "Aurangabad (Bihar)"
    if ((state_name=="maharashtra") and (district_name=="Aurangabad")):
        district_name = "Aurangabad (Maharashtra)"
    if ((state_name=="Maharashtra") and (district_name=="Aurangabad")):
        district_name = "Aurangabad (Maharashtra)"

    if ((state_name=="karnataka") and (district_name=="Bijapur")):
        district_name = "Bijapur (Karnataka)"
    if ((state_name=="Karnataka") and (district_name=="Bijapur")):
        district_name = "Bijapur (Karnataka)"
    if ((state_name=="chhattisgarh") and (district_name=="Bijapur")):
        district_name = "Bijapur (Chhattisgarh)"
    if ((state_name=="Chhattisgarh") and (district_name=="Bijapur")):
        district_name = "Bijapur (Chhattisgarh)"

    if ((state_name=="maharashtra") and (district_name=="Raigarh")):
        district_name = "Raigarh (Maharashtra)"
    if ((state_name=="Maharashtra") and (district_name=="Raigarh")):
        district_name = "Raigarh (Maharashtra)"
    if ((state_name=="chhattisgarh") and (district_name=="Raigarh")):
        district_name = "Raigarh (Chhattisgarh)"
    if ((state_name=="Chhattisgarh") and (district_name=="Raigarh")):
        district_name = "Raigarh (Chhattisgarh)"
    
    if ((state_name=="himachalpradesh") and (district_name=="Bilaspur")):
        district_name = "Bilaspur (Himachal Pradesh)"
    if ((state_name=="Himachal Pradesh") and (district_name=="Bilaspur")):
        district_name = "Bilaspur (Himachal Pradesh)"
    if ((state_name=="chhattisgarh") and (district_name=="Bilaspur")):
        district_name = "Bilaspur (Chhattisgarh)"
    if ((state_name=="Chhattisgarh") and (district_name=="Bilaspur")):
        district_name = "Bilaspur (Chhattisgarh)"
    
    if ((state_name=="uttarpradesh") and (district_name=="Balrampur")):
        district_name = "Balrampur (Uttar Pradesh)"
    if ((state_name=="Uttar Pradesh") and (district_name=="Balrampur")):
        district_name = "Balrampur (Uttar Pradesh)"
    if ((state_name=="chhattisgarh") and (district_name=="Balrampur")):
        district_name = "Balrampur (Chhattisgarh)"
    if ((state_name=="Chhattisgarh") and (district_name=="Balrampur")):
        district_name = "Balrampur (Chhattisgarh)"

    if ((state_name=="uttarpradesh") and (district_name=="Hamirpur")):
        district_name = "Hamirpur (Uttar Pradesh)"
    if ((state_name=="Uttar Pradesh") and (district_name=="Hamirpur")):
        district_name = "Hamirpur (Uttar Pradesh)"
    if ((state_name=="himachalpradesh") and (district_name=="Hamirpur")):
        district_name = "Hamirpur (Himachal Pradesh)"
    if ((state_name=="Himachal Pradesh") and (district_name=="Hamirpur")):
        district_name = "Hamirpur (Himachal Pradesh)"
    
    if ((state_name=="uttarpradesh") and (district_name=="Pratapgarh")):
        district_name = "Pratapgarh (Uttar Pradesh)"
    if ((state_name=="Uttar Pradesh") and (district_name=="Pratapgarh")):
        district_name = "Pratapgarh (Uttar Pradesh)"
    if ((state_name=="rajasthan") and (district_name=="Pratapgarh")):
        district_name = "Pratapgarh (Rajasthan)"
    if ((state_name=="Rajasthan") and (district_name=="Pratapgarh")):
        district_name = "Pratapgarh (Rajasthan)"


    return district_name
  

### Derive status for raw data 1 and raw data 2 -- One time code

In [120]:
"""
df1 = pd.read_csv("../data/"+folder+"/raw_data1.csv")
df1.dropna(subset=["Date Announced"], inplace=True)

df2 = pd.read_csv("../data/"+folder+"/raw_data2.csv")
df2.dropna(subset=["Date Announced"], inplace=True)

df = df1.append(df2)

df.rename(columns={"Contracted from which Patient (Suspected)":"contracted_from", "Current Status":"status", "Notes":"notes",
                   "Detected City":"city", "Detected District":"district", "Detected State": "state","State code":"state_code",
                   "Date Announced":"date", "Type of transmission":"transmission_type", "Age Bracket":"age", "Gender":"gender",
                   "State Patient Number": "state_pid", "Patient Number":"pid", "Status Change Date":"status_change_date"}, inplace=True)
df = df[["pid", "date", "city", "district", "state", "state_code", "contracted_from", "status", "transmission_type", "notes", "status_change_date"]]
df["date"] = pd.to_datetime(df["date"], dayfirst=True)
df["status_change_date"] = pd.to_datetime(df["status_change_date"], dayfirst=True)
df["status_change_date"] = df["date"]
df["derived_status"] = "Hospitalized"
df["is_derived_status_updated"] = 0

df.reset_index(inplace=True)
del df["index"]

dfsd = pd.read_csv("../data/"+folder+"/statewise_daily_deceased.csv")
del dfsd["TT"]
dfsd["date"] = pd.to_datetime(dfsd["date"], dayfirst=True)
dfsd

dfsr = pd.read_csv("../data/"+folder+"/statewise_daily_recovered.csv")
del dfsr["TT"]
dfsr["date"] = pd.to_datetime(dfsr["date"], dayfirst=True)

dfr = df.copy()

num_recovered = 0
num_deceased = 0

for col in dfsd.columns:
    start_date = pd.datetime(2020,3,14)
    end_date = pd.datetime(2020,4,26)
    current_date = start_date
    print(col)

    if col!="date":
        state_code = col
        
        while(current_date<=end_date):
            
            num_deaths_state_on_day = int(dfsd[(dfsd["date"]==current_date)][state_code].values[0])
            num_recoveries_state_on_day = int(dfsr[(dfsr["date"]==current_date)][state_code].values[0])
            
            if num_deaths_state_on_day>0:
                idxs = dfr[(dfr["date"]<current_date) & (dfr["state_code"]==state_code) & (dfr["is_derived_status_updated"]==0)].index
                if len(idxs)>0:
                    random_idx = idxs[random.sample(range(len(idxs)), num_deaths_state_on_day)]
                    for idx in random_idx:
                        dfr.at[dfr.index==idx, "status_change_date"] = current_date
                        dfr.at[dfr.index==idx, "derived_status"] = "Deceased"
                        dfr.at[dfr.index==idx, "is_derived_status_updated"] = 1
                        num_deceased += 1
            
            if num_recoveries_state_on_day>0:
                idxs = dfr[(dfr["date"]<current_date) & (dfr["state_code"]==state_code) & (dfr["is_derived_status_updated"]==0)].index
                if len(idxs)>0:
                    random_idx = idxs[random.sample(range(len(idxs)), num_recoveries_state_on_day)]
                    for idx in random_idx:
                        dfr.at[dfr.index==idx, "status_change_date"] = current_date
                        dfr.at[dfr.index==idx, "derived_status"] = "Recovered"
                        dfr.at[dfr.index==idx, "is_derived_status_updated"] = 1
                        num_recovered += 1
                
            current_date += timedelta(1)
dfr.to_csv("../data/06052020/raw_data1_data2_withStatusMapped.csv", index=False)
"""
print("One time code")

One time code


In [ ]:
### full_data_patient_level_withoutDistrictMapped_withoutStatusMapped
"""
df = pd.read_csv("../data/06052020/one_time/raw_data1_data2_withStatusMapped.csv")
del df["status"]
df.rename(columns={"derived_status":"status"}, inplace=True)
df["num_cases"] = 1
df = df[["pid", "date", "city", "district", "state", "state_code", "contracted_from", "status", "transmission_type", "notes", "status_change_date", "is_derived_status_updated", "num_cases"]]

df3 = pd.read_csv("../data/"+folder+"/raw_data3.csv")
df3.rename(columns={"Contracted from which Patient (Suspected)":"contracted_from", "Current Status":"status", "Notes":"notes",
                   "Detected City":"city", "Detected District":"district", "Detected State": "state","State code":"state_code",
                   "Date Announced":"date", "Type of transmission":"transmission_type", "Age Bracket":"age", "Gender":"gender",
                   "State Patient Number": "state_pid", "Patient Number":"pid", "Status Change Date":"status_change_date",
                   "Num Cases":"num_cases"}, inplace=True)
df3["is_derived_status_updated"] = 0
df3 = df3[["pid", "date", "city", "district", "state", "state_code", "contracted_from", "status", "transmission_type", "notes", "status_change_date", "is_derived_status_updated", "num_cases"]]
df3.dropna(subset=["date", "state"], inplace=True)
df3h = df3[df3["status"] == "Hospitalized"].copy()

dfh = df3h.copy()
dfh.sort_values("num_cases",ascending=False, inplace=True)
dfh.reset_index(inplace=True)
del dfh["index"]
dfh

dfr = df.copy()
dfr.reset_index(inplace=True)
del dfr["index"]
#dfr = pd.DataFrame(columns=dfh.columns)

idx=0
ncases = 0

for i in range(len(dfh)):
    if i%500==0:
        print(i)
    current_num_cases = dfh.iloc[i]["num_cases"]
    state_name = dfh.iloc[i]["state"]
    district_name = dfh.iloc[i]["district"]

    if ( (current_num_cases>0) and (current_num_cases==current_num_cases) ):
        current_num_cases = int(current_num_cases)
        dfr = dfr.append([dfh.iloc[i]]*current_num_cases, ignore_index=True)
        ncases += current_num_cases

    elif ( (current_num_cases<0) and (current_num_cases==current_num_cases) ):
        current_num_cases = int(current_num_cases)
        
        if (district_name==district_name):
            if len(dfr[ (dfr["district"]==district_name) & (dfr["state"]==state_name) & (dfr["status"]=="Hospitalized")])>=abs(current_num_cases):
                dft = dfr[ (dfr["district"]==district_name) & (dfr["state"]==state_name) & (dfr["status"]=="Hospitalized")]
            else:
                dft = dfr[ (pd.isnull(dfr["district"])) & (dfr["state"]==state_name) & (dfr["status"]=="Hospitalized")]
        else:
            dft = dfr[ (pd.isnull(dfr["district"])) & (dfr["state"]==state_name) ]
        del_index_list = dft.index[np.random.choice(range(0,len(dft)), abs(current_num_cases), replace=False)]
        dfr.drop(dfr.index[del_index_list], inplace=True)
        ncases += current_num_cases

    if ((dfh[0:i+1]["num_cases"].sum())+ len(df))!=len(dfr):
        print("-----"+str(i)+"------")
        
dfr["num_cases"] = 1
dfr.to_csv("../data/09052020/one_time/full_data_patient_level_withoutDistrictMapped_withoutStatusMapped.csv", index=False)
"""
print("one time")

In [ ]:
"""
df3 = pd.read_csv("../data/09052020/raw_data3.csv")
df3.rename(columns={"Contracted from which Patient (Suspected)":"contracted_from", "Current Status":"status", "Notes":"notes",
                   "Detected City":"city", "Detected District":"district", "Detected State": "state","State code":"state_code",
                   "Date Announced":"date", "Type of transmission":"transmission_type", "Age Bracket":"age", "Gender":"gender",
                   "State Patient Number": "state_pid", "Patient Number":"pid", "Status Change Date":"status_change_date",
                   "Num Cases":"num_cases"}, inplace=True)
df3["is_derived_status_updated"] = 0
df3 = df3[["pid", "date", "city", "district", "state", "state_code", "contracted_from", "status", "transmission_type", "notes", "status_change_date", "is_derived_status_updated", "num_cases"]]
df3.dropna(subset=["date", "state"], inplace=True)
df3nh = df3[df3["status"]!="Hospitalized"].copy()
df3nh["status_change_date"] = df3nh["date"]
df3nh.reset_index(inplace=True)
del df3nh["index"]

df = pd.read_csv("../data/09052020/one_time/full_data_patient_level_withoutDistrictMapped_withoutStatusMapped.csv")

for i in range(len(df3nh)):
    current_date = df3nh.iloc[i]["date"]
    status = df3nh.iloc[i]["status"]
    state_name = df3nh.iloc[i]["state"]
    district_name = df3nh.iloc[i]["district"]
    num_status_update = df3nh.iloc[i]["num_cases"]

    if (num_status_update==num_status_update) and (num_status_update>0):
        dft = df[(df["state"]==state_name) & (df["district"]==district_name) & (df["status"]=="Hospitalized")]
        if len(dft) < num_status_update:
            dft = df[(df["state"]==state_name) & (pd.isnull(df["district"])) & (df["status"]=="Hospitalized")]
        if len(dft)>0:
            update_index_list = dft.index[np.random.choice(range(0,len(dft)), min(len(dft), int(num_status_update)), replace=False)]
            for idx in update_index_list:
                df.at[df.index==idx, "status_change_date"] = current_date
                df.at[df.index==idx, "status"] = status
                df.at[df.index==idx, "is_derived_status_updated"] = 1
    
    elif (num_status_update==num_status_update) and (num_status_update<0):
        dft = df[(df["state"]==state_name) & (df["district"]==district_name) & (df["status"]==status)]
        if len(dft) < abs(num_status_update):
            dft = df[(df["state"]==state_name) & (pd.isnull(df["district"])) & (df["status"]==status)]
        if len(dft)>0:
            del_index_list = dft.index[np.random.choice(range(0,len(dft)), min(len(dft), abs(int(num_status_update))), replace=False)]
            df.drop(del_index_list, inplace=True)
            
df.to_csv("../data/09052020/one_time/full_data_patient_level_withoutDistrictMapped_withStatusMapped.csv")
"""
"""
df = pd.read_csv("../data/09052020/one_time/full_data_patient_level_withoutDistrictMapped_withStatusMapped.csv")
del df["Unnamed: 0"]
df["date"] = pd.to_datetime(df["date"], dayfirst=True)
df["status_change_date"] = pd.to_datetime(df["status_change_date"], dayfirst=True)
df.to_csv("../data/09052020/one_time/full_data_patient_level_withoutDistrictMapped_withStatusMapped.csv", index=False)
"""
print("One time")

In [ ]:
"""
df = pd.read_csv("../data/09052020/one_time/full_data_patient_level_withoutDistrictMapped_withStatusMapped.csv")
df.at[ (pd.isnull(df["district"])) & (df["state"]=="Delhi"), "district" ] = "Delhi"

dfd = pd.read_csv("../data/"+folder+"/derived/district_confirmed_active_recovered_deceased_population_casespm.csv")
state_list = dfd["state"].unique()

for st in state_list:
    if st not in ["Delhi"]:
        dfds = dfd[dfd["state"]==st].copy()
        dfds.sort_values("confirmed", ascending=False)
        district_list = dfds["district"].unique()

        for dt in district_list:
            district_num_confirmed = dfd[dfd["district"]==dt]["confirmed"].values[0]
            district_num_mapped = len(df[(df["state"]==st) & (df["district"]==dt)])
            
            if (district_num_confirmed > district_num_mapped):
                num_missing_mapped = int(district_num_confirmed - district_num_mapped)
                
                if len(df[ (pd.isnull(df["district"])) & (df["state"]==st) ])>0:
                    dft = df[ (pd.isnull(df["district"])) & (df["state"]==st) ]
                    selected_idx = dft.index[np.random.choice(range(0, len(dft)), min(len(dft), num_missing_mapped), replace=False)]
                    
                    for idx in selected_idx:
                        df.at[df.index==idx, "district"] = dt

dfunmapped = df[pd.isnull(df["district"])].copy()
for i in range(len(dfunmapped)):
    state_name = df.iloc[i]["state"]
    dfds = dfd[dfd["state"]==st].copy()
    missing_cases_mapped_to_district_name = dfds.sort_values("confirmed", ascending=False).head(1)["district"].values[0]
    
    idx = dfunmapped.index[i]
    df.at[df.index==idx, "district"] = missing_cases_mapped_to_district_name

df.to_csv("../data/09052020/one_time/full_data_patient_level_withDistrictMapped_withStatusMapped.csv", index=False)
"""
"""
df = pd.read_csv("../data/09052020/one_time/full_data_patient_level_withDistrictMapped_withStatusMapped.csv")
df["date"] = pd.to_datetime(df["date"], dayfirst=True).dt.date
df["status_change_date"] = pd.to_datetime(df["status_change_date"], dayfirst=True).dt.date
df.at[(df["district"]=="Italians*")&(df["state"]=="Haryana"), "district"] = "Gurugram"
df.at[(df["district"]=="Italians*")&(df["state"]=="Rajasthan"), "district"] = "Jaipur"
df.to_csv("../data/09052020/one_time/full_data_patient_level_withDistrictMapped_withStatusMapped.csv", index=False)
df
"""

In [ ]:
"""
### Get list of all districts from map
dfmd = pd.DataFrame()
idx = 0
for fp in os.listdir("../data/state_geojson_raw/"):
    if not fp.startswith(".") and fp.endswith(".json"):
        state_name = fp.split(".")[0]
        state_name_mapped = state_name_mapping[state_name]
        state_name_list.append(state_name)
        #print(state_name)

        with open("../data/state_geojson_raw/"+fp) as topodata:
            raw_data = json.load(topodata)

        data = raw_data
        district_list = data["objects"][state_name+"_district"]["geometries"]
        
        for i in range(len(district_list)):
            district_name = data["objects"][state_name+"_district"]["geometries"][i]["properties"]["district"]
        
            dfmd.at[idx, "district"] = district_name
            dfmd.at[idx, "state"] = state_name_mapped
            idx+=1
dfmd.to_csv("../data/district_list_map.csv", index=False)
"""
print("One time code")